<a href="https://colab.research.google.com/github/Takenoko0/Japanese-Tongue-twister-Generator/blob/main/Japanese%20Tongue%20Twister.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import random
import os
import tempfile
import time
import glob
from gtts import gTTS
from pydub import AudioSegment
from IPython.display import Audio, display

#############################################################
#                    ユーザー設定部分                       #
#############################################################

# 基本設定
OUTPUT_COUNT = 1000  # 生成する早口言葉の数
DIFFICULTY_LEVEL = "普通"  # "簡単", "普通", "難しい", "超難しい", "鬼畜級"
DIFFICULTY_LEVELS = ["簡単", "普通", "難しい", "超難しい", "鬼畜級"]  # 難易度レベル（比較用）
SELECTED_DIALECTS = ["博多弁", "関西弁", "津軽弁", "沖縄方言", "東北弁"]  # 使用する方言
GENERATION_MODE = "standard"  # 生成モード: "standard", "dialect_only", "classic_only", "sound_patterns", "row_column", "meaningful", "word_rules"
ENABLE_AUTO_GENERATE = True  # 自動生成機能（追加パターン）を有効にするか

# 音声生成設定
SPEECH_SETTINGS = {
    "enable": False,          # 音声生成の有効/無効
    "default_speed": 1.5,    # 再生速度（通常の何倍か）
    "play_in_colab": True    # Google Colab上で再生するか
}

# 自動生成時の各パターン比率
PATTERN_WEIGHTS = {
    "adj_noun_pattern": 0.1,          # 形容詞+名詞パターン
    "noun_list_pattern": 0.1,         # 名詞列挙パターン
    "noun_particle_verb_pattern": 0.1, # 名詞+助詞+動詞パターン
    "place_time_pattern": 0.1,        # 場所+時間+動詞パターン
    "adverb_verb_pattern": 0.125,       # 副詞+動詞パターン
    "season_adj_noun_pattern": 0.125,   # 季節+形容詞+名詞パターン
    "animal_food_pattern": 0.125,       # 動物+食べ物+動詞パターン
    "combined_pattern": 0,          # 既存パターン組み合わせ
    "row_column_pattern": 0.125,        # 行・段パターン
    "meaningful_pattern": 0.125,        # 意味のある文パターン
    "word_rule_pattern": 0.125          # 単語ルールパターン
}

#############################################################
#                    データ定義部分                         #
#############################################################

# ※※※ ここから単語データ部分（後で詳細なデータに置き換え） ※※※

# 基本の早口言葉（古典的なもの）- サンプルデータのみ
CLASSIC_TWISTERS = [
"隣の客はよく柿食う客だ",
"東京特許許可局",
"バスガス爆発",
"赤巻紙青巻紙黄巻紙",
"生麦生米生卵",
"坊主が屏風に上手に坊主の絵を描いた",
"新春シャンソンショー",
"この釘はひきぬきにくい釘だ",
"かえるぴょこぴょこみぴょこぴょこ",
"スモモも桃も桃のうち",
"アンドロイドロイドロイド",
"老若男女の着物の布",
"貨客船と回転寿司",
"紫式部の数珠",
"コーラとグローブとクローバー",
"テストステロンと危機一髪",
"五条の橋で小猿が子猫を小突いて小糠に小判",
"回文で高速増殖炉もんじゅ",
"きゃりーぱみゅぱみゅのブルドーザー土砂",
"トンネルを抜けるとそこは雪国だった",
"青巻紙赤巻紙黄巻紙のクラッカー",
"インド人とシンド人と日本人のおじいちゃん",
"ジェットコースターでがらがらからから",
"カエルが飲み込んだ風船と風船をとばしたカエル",
"木木曜日にこぶしをにぎりしめしめとにぎりしめた",
"魔法の粉の粉の魔法で寿限無寿限無",
"五劫のすりきれ海砂利水魚の水行末雲来末風来末",
"にわにわにわにわとりがいる",
"ブラックベリーとラズベリーとブルーベリーとクランベリー",
"パブロ・ディエゴ・ホセ・フランシスコ・デ・パウラ・フアン・ネポムセーノ",
"シェシェセッションであぶらあぶら油あぶら",
"総括責任者と図書館司書",
"あかまきがみあおまきがみきまきがみ",
"ぶらぶらブラブラからかさかして",
"すもももももももものうち",
"とっきょきょかきょくちょうきょきょきょきょきょきょきょきょ",
"ろくろくろくろあおいあおいあおい",
"きゃきゅきょきゃきゅきょじゅげむじゅげむ",
"たけやぶやけたなまむぎなまごめなまたまご",
"はたはたはたはたはたはたほたるのひかり",
"まきまきまきまきまきまきみどりのまきがみ",
"あかいまきがみきいろいまきがみむらさきしきぶ",
"もみじまんじゅうとやきにくやきとり",
"よもぎもちもちもちもちわかめわかめわかめ",
"いすずのいすいすずのいすうぐいすうぐいす",
"えんぴつけずりけずりえんぴつおにぎりおにぎり",
"かたかなカタカナきつつききつつき",
"くるくるくるくるくるくるけいさつけいさつ",
"こうこうこうこうこうこうさくらさくさくらさく",
"しんぶんしんぶんしんぶんすいかすいかすいか",
"せんせいせんせいせんせいそうそうそうそう",
"たこやきたこやきたこやきちりとりちりとり",
"つるつるつるつるつるつるてるてるぼうず",
"とんとんとんとんとんとんなんばんなんばん",
"にんじんにんじんにんじんぬるぬるぬるぬる",
"ねこねこねこねこねこねこのりのりのりのり",
"はしはしはしはしはしはしひらがなひらがな",
"ふわふわふわふわふわふわへびへびへびへび",
"ほしほしほしほしほしほしまぜまぜまぜまぜ",
"みずみずみずみずみずみずむしむしむしむし",
"めがねめがねめがねめがねもりもりもりもり",
"やきそばやきそばやきそばゆきゆきゆきゆき",
"よるよるよるよるよるよるらーめんらーめん",
"りんごりんごりんごりんごるんるんるんるん",
"れもんれもんれもんれもんろうそくろうそく",
"わさびわさびわさびわさびがっこうがっこう",
"ぎょうざぎょうざぎょうざぐるぐるぐるぐる",
"げんきげんきげんきげんきごはんごはんごはん",
"ざぶとんざぶとんざぶとんじてんしゃじてんしゃ",
"ずんずんずんずんずんずんぜんぶぜんぶぜんぶ",
"ぞうさんぞうさんぞうさんだるまだるまだるま",
"ぢょうぎぢょうぎぢょうぎづくしづくしづくし",
"でんわでんわでんわでんわどんぶりどんぶり",
"ばななばななばななばななびりびりびりびり",
"ぶたぶたぶたぶたぶたぶたべんきょうべんきょう",
"ぼうしぼうしぼうしぼうしぱんだぱんだぱんだ",
"ぴかぴかぴかぴかぴかぴかぷりんぷりんぷりん",
"ぺんぺんぺんぺんぺんぺんぽんぽんぽんぽん",
"きょうりゅうきょうりゅうぎゅうにゅうぎゅうにゅう",
"しゃしんしゃしんしゃしんじゅうどうじゅうどう",
"ちょうちょちょうちょちょうちょにゅうがくにゅうがく",
"ひゃくえんひゃくえんひゃくえんびょういんびょういん",
"みょうじみょうじみょうじりょうりりょうりりょうり",
"ぎょうざのぎょうざのぎょうざのぎょうざ",
"きゅうりのきゅうりのきゅうりのきゅうり",
"しゃべりすぎてしゃべりすぎてしゃべりすぎて",
"じゅうじゅうじゅうじゅうじゅうじゅうじゅうじゅう",
"ちゅうしゃちゅうしゃちゅうしゃにゅうようじにゅうようじ",
"ひゃっかじてんひゃっかじてんびょうきのびょうき",
"みょうがのみょうがのみょうがりょうこうりょうこう",
"きょうのきょうはきょうのきょうぎゅうにくぎゅうにく",
"しゅくだいしゅくだいしゅくだいじゅんびじゅんび",
"ちょきんちょきんちょきんにゅうがくしけんにゅうがくしけん",
"ひょうげんひょうげんひょうげんびょうどうびょうどう",
"みょうにちみょうにちみょうにちりょうしんりょうしん",
"きょうかいきょうかいきょうかいぎゅうどんぎゅうどん",
"しゅっせきしゅっせきしゅっせきじゅうたんじゅうたん",
"ちょうちょうちょうちょうちょうちょうにゅうしゃしけんにゅうしゃしけん",
"ひょうしきひょうしきひょうしきびょうにんびょうにん",
"みょうじょうみょうじょうみょうじょうりょうきんりょうきん",
"きょうしつきょうしつきょうしつぎゅうなべぎゅうなべ",
"しゅっぱつしゅっぱつしゅっぱつじゅうみんじゅうみん",
"ちょうないちょうないちょうないにゅうじょうけんにゅうじょうけん",
"ひょうろんひょうろんひょうろんびょうしつびょうしつ",
"みょうごにちみょうごにちみょうごにちりょうてりょうて",
"きょうだいきょうだいきょうだいぎゅうひぎゅうひ",
"しゅっちょうしゅっちょうしゅっちょうじゅうしょじゅうしょ",
"ちょうめいちょうめいちょうめいにゅうこくにゅうこく",
"ひょうばんひょうばんひょうばんびょうきびょうき",
"みょうにくみょうにくみょうにくりょうりつりょうりつ",
"あかいりんごとあおいりんご",
"かたいかきとやわらかいかき",
"さくらんぼとすもものなるき",
"たけのこのてっぺんとたぬきのきんたま",
"なまけものとかめのかけっこ",
"はなのあなからなにがでるか",
"まつのきとうめのきのあいだ",
"やまのふもとのふもとのやま",
"わらいながらわらわれながら",
"がらがらへびがからからへび",
"ざらざらいしでじゃりじゃりいわ",
"だんだらちょうちょがちょうだらちょうちょ",
"ばばばあちゃんのまごのぱぱ",
"ぱんだがぱんをぱくぱくたべる",
"あさあけあかねいろあおぞら",
"いそいそいそがしいいそぎんちゃく",
"うきうきわくわくどきどきはらはら",
"えんえんえんどれすえんとつ",
"おいおいおちょこおいしゃさん",
"かっかかっかてんかだいかっか",
"きらきらひかるほしのひかり",
"くるくるまわるくるりんぱ",
"けろけろけろっぴとげろげろ",
"こんこんちきのこんこんちきちき",
"さらさらさらえてさっぱりした",
"しゅるしゅるしゅっしゅっしゅわしゅわ",
"すやすやすいみんすいすいすい",
"せかせかいそいでせっせとする",
"そわそわどきどきわくわくする",
"たらたらたれてたまらないたれ",
"ちくたくちくたくとけいのおと",
"つるつるすべすべつやつやぴか",
"てくてくあるいてとことこいく",
"とぼとぼあるいてとことこいく",
"にこにこにっこりにやにやする",
"ぬくぬくあったかぬっくぬく",
"ねんねんころりよおころりよ",
"のろのろゆっくりのんびりと",
"ぱくぱくもぐもぐごくごくごっくん",
"ひらひらひらめくひらひらちょう",
"ふわふわこわれやすいふうせん",
"へとへとくたくたへろへろだ",
"ほかほかあったかほくほくいも",
"まざまざみえるまぼろしのまち",
"みるみるうちにみるみるみえる",
"むしゃむしゃもぐもぐむしゃむしゃ",
"めそめそなかないでめげないで",
"もじもじもぢもぢもじもじする",
"やきもきどきどきはらはらする",
"ゆらゆらゆれるゆらゆらゆり",
"よたよたよろよろよちよちあるく",
"らんらんらんまんえがおでらんらん",
"りんりんりんとなるすずのね",
"るんるんるんぱっぱるんるんるん",
"れろれろぺろぺろれろれろする",
"ろんろんろんぱっぱろんろんろん",
"わんわんわおーんわんわんわん",
"ぎゅーぎゅーぎゅっとぎゅうにゅう",
"ぐるぐるまわってぐるぐるぐる",
"げらげらげらげらわらいころげる",
]

# 難読熟語・言いにくい言葉 - サンプルデータのみ
DIFFICULT_PHRASES = [

"総括責任者",
"図書館司書",
"抗張力試験機",
"突発性難聴",
"無軌道輸送",
"一挙一動",
"確率統計学",
"合法的行為",
"特殊相対性理論",
"量子力学的解釈",
"超電導現象",
"非線形動力学",
"複素関数論",
"微分幾何学",
"トポロジー的空間",
"エルゴード理論",
"カオス力学系",
"フラクタル次元",
"リーマン予想",
"ゼータ関数",
"フェルマーの最終定理",
"ポアンカレ予想",
"ナヴィエ・ストークス方程式",
"ヤング図形",
"モジュラー形式",
"楕円曲線暗号",
"量子コンピューティング",
"人工知能倫理",
"機械学習アルゴリズム",
"ニューラルネットワーク",
"深層強化学習",
"自然言語処理",
"コンピュータビジョン",
"エッジコンピューティング",
"ブロックチェーン技術",
"量子暗号通信",
"サイバーセキュリティ",
"生体認証システム",
"仮想現実技術",
"拡張現実応用",
"自動運転技術",
"ドローン制御",
"宇宙エレベーター",
"核融合発電",
"超伝導リニア",
"光量子コンピュータ",
"脳機械インターフェース",
"遺伝子編集技術",
"幹細胞再生医療",
"ナノテクノロジー",
"メタマテリアル",
"超臨界流体",
"プラズマ物理学",
"暗黒物質探索",
"重力波観測",
"宇宙線観測",
"ニュートリノ振動",
"素粒子加速器",
"超対称性理論",
"弦理論",
"ループ量子重力理論",
"ホログラフィック原理",
"ブラックホール情報パラドックス",
"宇宙インフレーション理論",
"多世界解釈",
"時間の矢",
"量子もつれ",
"量子テレポーテーション",
"量子暗号",
"量子センシング",
"量子メタロジー",
"トポロジカル量子計算",
"アニオン統計",
"マヨラナ粒子",
"トポロジカル絶縁体",
"ワイル半金属",
"高温超伝導体",
"スピントロニクス",
"フォトニック結晶",
"メタサーフェス",
"プラズモニクス",
"ナノフォトニクス",
"バイオフォトニクス",
"オプトジェネティクス",
"神経可塑性",
"エピジェネティクス",
"プロテオミクス",
"メタボロミクス",
"システム生物学",
"合成生物学",
"バイオインフォマティクス",
"計算生物学",
"分子動力学シミュレーション",
"第一原理計算",
"モンテカルロ法",
"有限要素法",
"境界要素法",
"分子軌道法",
"密度汎関数理論",
"量子化学計算",
"分子設計",
"薬物動態学",
"構造生物学",
"タンパク質工学",
"抗体工学",
"遺伝子治療",
"細胞治療",
"免疫療法",
"がんゲノム医療",
"精密医療",
"再生医療",
"オルガノイド",
"バイオプリンティング",
"ナノ医療",
"テラヘルツ波",
"アト秒科学",
"超高速分光",
"単一分子分光",
"超解像顕微鏡",
"クライオ電子顕微鏡",
"中性子散乱",
"シンクロトロン放射光",
"自由電子レーザー",
"核磁気共鳴分光法",
"質量分析法",
"クロマトグラフィー",
"電気化学分析",
"バイオセンサー",
"マイクロ流体デバイス",
"ラボオンチップ",
"マイクロ電気機械システム",
"ナノエレクトロメカニカルシステム",
"スマートマテリアル",
"自己修復材料",
"形状記憶合金",
"圧電材料",
"熱電材料",
"超撥水材料",
"生分解性プラスチック",
"カーボンナノチューブ",
"グラフェン",
"二次元材料",
"トポロジカル材料",
"準結晶",
"メタマテリアル",
"フォトニック結晶",
"コロイド結晶",
"ソフトマター",
"液晶",
"高分子",
"ゲル",
"エマルション",
"ミセル",
"界面活性剤",
"超臨界流体",
"イオン液体",
"プラズマ",
"磁性流体",
"強相関電子系",
"量子ホール効果",
"トポロジカル超伝導",
"マヨラナ粒子",
"スピン流",
"スキルミオン",
"マグノン",
"フォノン",
"ポラリトン",
"エキシトン",
"プラズモン",
"フォトニック分子",
"量子ドット",
"量子井戸",
"量子細線",
"ヘテロ構造",
"エピタキシャル成長",
"原子層堆積",
"分子線エピタキシー",
"パルスレーザー堆積",
"スパッタリング",
"化学気相成長",
"物理気相成長",
"電気化学堆積",
"ゾル-ゲル法",
"水熱合成",
"マイクロ波合成",
"超音波合成",
"メカノケミカル合成",
"プラズマ合成",
"レーザーアブレーション",
"イオンビーム合成",
"電子ビーム蒸着",
"分子自己組織化",
"ナノ粒子合成",
"量子ドット合成",
"コアシェルナノ粒子",
"異方性ナノ粒子",
"プラズモニックナノ粒子",
"磁性ナノ粒子",
"アップコンバージョンナノ粒子",
"量子カスケードレーザー",
"面発光レーザー",
"テラヘルツ量子カスケードレーザー",
"アト秒パルスレーザー",
"超短パルスレーザー",
"ファイバーレーザー",
"ランダムレーザー",
"プラズモニックレーザー",
"トポロジカルレーザー",
"量子もつれ光源",
"単一光子源",
"もつれ光子対源",
"スクイーズド光",
"非線形光学",
"光周波数コム",
"光格子時計",
"量子メモリ",
"量子中継器",
"量子インターネット"
]

# 方言の早口言葉 - サンプルデータのみ
DIALECT_TWISTERS = {

"博多弁": [
"ばってんばってん、ばってんがばってんたい、ばってんばってんばってんよ",
"めんたいこたべたばい、めんたいこうまかばい、めんたいこほしかね",
"ごっつぉごっつぉ、ごっつぉたいね、ごっつぉしよるばい",
"なんばしよっと、なんばしよっとね、なんばしよっとやろうか",
"がばいがばい、がばいうまかね、がばいたまらんばい",
"ちゃんぽんたべたか、ちゃんぽんうまかろ、ちゃんぽんくうたい",
"もつ鍋もつ鍋、もつ鍋たいな、もつ鍋くおうや",
"ばりよかばい、ばりよかたい、ばりよかとよ",
"うどんうどん、うどんたいね、うどんくおうや",
"しこたましこたま、しこたまたいね、しこたまくうたい",
"ごうついごうつい、ごうついたいね、ごうついばってん",
"めっちゃめっちゃ、めっちゃたいね、めっちゃよかばい",
"やかましかやかましか、やかましかたいね、やかましかばってん",
"だごんだごん、だごんたいね、だごんくおうや",
"うまかもんうまかもん、うまかもんたいね、うまかもんくうたい",
"ぼっけもんぼっけもん、ぼっけもんたいね、ぼっけもんばってん",
"なんしよっとなんしよっと、なんしよっとたいね、なんしよっとやろうか",
"ぬくいぬくい、ぬくいたいね、ぬくいばってん",
"いごっつぉういごっつぉう、いごっつぉうたいね、いごっつぉうばってん",
"ごたごたごたごた、ごたごたたいね、ごたごたしよるばい"
],
"関西弁": [
"ほんまかいな、ほんまかいなぁ、ほんまかいなほんま",
"なんでやねん、なんでやねんな、なんでやねんねん",
"あかんで、あかんでぇ、あかんでほんま",
"めっちゃ、めっちゃやで、めっちゃめっちゃやねん",
"ちゃうちゃう、ちゃうちゃうねん、ちゃうちゃうちゃうで",
"せやかて、せやかてぇ、せやかてほんま",
"おおきに、おおきにな、おおきにおおきに",
"ぼちぼち、ぼちぼちやな、ぼちぼちいこか",
"しゃあない、しゃあないわ、しゃあないしゃあない",
"けったいな、けったいなぁ、けったいなけったい",
"ちゃうねん、ちゃうねんで、ちゃうねんちゃう",
"あほか、あほかいな、あほかあほあほ",
"ほんまもんや、ほんまもんやで、ほんまもんやねん",
"なんぼなん、なんぼなんや、なんぼなんぼ",
"えらいこっちゃ、えらいこっちゃな、えらいこっちゃえらい",
"ぼけぇ、ぼけぇな、ぼけぇぼけぼけ",
"あかん、あかんわ、あかんあかんあかん",
"ちゃうねんけど、ちゃうねんけどな、ちゃうねんけどちゃう",
"ほんまやで、ほんまやでぇ、ほんまやでほんま",
"なんでやねんな、なんでやねんなぁ、なんでやねんなんで"
],
"津軽弁": [
"なんだばや、なんだばやぁ、なんだばやなんだ",
"めんこい、めんこいなぁ、めんこいめんこ",
"けっぱれ、けっぱれな、けっぱれけっぱれ",
"はんでがす、はんでがすな、はんでがすはんで",
"じょっぱり、じょっぱりだ、じょっぱりじょっぱり",
"わりかし、わりかしな、わりかしわり",
"なんぼがんばっても、なんぼがんばってもな、なんぼがんばってもなんぼ",
"だばだば、だばだばだ、だばだばだばだば",
"しゃっこい、しゃっこいな、しゃっこいしゃっこ",
"やっぱし、やっぱしな、やっぱしやっぱ",
"へばな、へばなぁ、へばなへば",
"なして、なしてだ、なしてなして",
"ごっつぉ、ごっつぉだ、ごっつぉごっつぉ",
"まだかや、まだかやぁ、まだかやまだ",
"めんこくて、めんこくてな、めんこくてめんこ",
"なんだって、なんだってな、なんだってなんだ",
"しったげ、しったげな、しったげしった",
"わがんね、わがんねな、わがんねわがん",
"めんこいべ、めんこいべな、めんこいべめんこ",
"なんだばしょ、なんだばしょな、なんだばしょなんだ"
],
"沖縄方言": [
"めんそーれ、めんそーれよー、めんそーれめんそーれ",
"ちゅらさん、ちゅらさんやー、ちゅらさんちゅら",
"にふぇーでーびる、にふぇーでーびるさー、にふぇーでーびるにふぇー",
"はいさい、はいさいぐすーよー、はいさいはいさい",
"うちなーんちゅ、うちなーんちゅやー、うちなーんちゅうちなー",
"あんまー、あんまーさー、あんまーあんまー",
"くぬ、くぬやー、くぬくぬ",
"まーさん、まーさんぐぁー、まーさんまーさん",
"ゆたさるぐとぅ、ゆたさるぐとぅよー、ゆたさるぐとぅゆたさる",
"うとぅとぅい、うとぅとぅいさー、うとぅとぅいうとぅ",
"やんばるんちゅ、やんばるんちゅやー、やんばるんちゅやんばる",
"なんくるないさ、なんくるないさー、なんくるないさなんくる",
"ちむどんどん、ちむどんどんやー、ちむどんどんちむ",
"いちゃりばちょーでー、いちゃりばちょーでーさー、いちゃりばちょーでーいちゃ",
"うちなーぐち、うちなーぐちやー、うちなーぐちうちなー",
"ゆいまーる、ゆいまーるさー、ゆいまーるゆい",
"くくる、くくるやー、くくるくくる",
"ちゅらかーぎー、ちゅらかーぎーさー、ちゅらかーぎーちゅら",
"まーさんぐぁー、まーさんぐぁーやー、まーさんぐぁーまーさん",
"あがりうまんちゅ、あがりうまんちゅやー、あがりうまんちゅあがり"
],
"東北弁": [
"んだずら、んだずらな、んだずらんだ",
"しったげ、しったげだ、しったげしった",
"めんこい、めんこいべ、めんこいめんこ",
"なじょした、なじょしたんだ、なじょしたなじょ",
"けっぱれ、けっぱれや、けっぱれけっぱれ",
"わらすこ、わらすこだべ、わらすこわらす",
"おめえ、おめえさ、おめえおめ",
"あづい、あづいなぁ、あづいあづ",
"しゃっこい、しゃっこいべ、しゃっこいしゃっこ",
"なんぼ、なんぼだべ、なんぼなんぼ",
"まんず、まんずいがべ、まんずまんず",
"おばんです、おばんですなぁ、おばんですおばん",
"えがった、えがったなぁ、えがったえがった",
"だばだば、だばだばだ、だばだばだば",
"ずーずー、ずーずーべん、ずーずーずー",
"おだづなよ、おだづなよな、おだづなよおだづ",
"がっこ、がっこさいぐべ、がっこがっこ",
"おがる、おがるべ、おがるおがる",
"まげる、まげるな、まげるまげ",
"こわい、こわいべ、こわいこわ"
]
}

# 五十音の行ごとの単語グループ - サンプルデータのみ
ROW_WORDS = {
    "あ行": ["あめ", "朝", "足", "青", "赤", "秋", "遊び", "嵐", "明かり", "あくび",
      "挨拶", "あご", "頭", "あばら", "悪魔", "あじ", "あいこ", "汗", "泡", "油",
      "網", "甘み", "あさって", "あした", "あそこ", "小豆", "あなた", "朝日", "雨降り", "赤ちゃん",
      "相手", "青空", "秋風", "朝ご飯", "雨雲", "穴埋め", "歩み", "朝霧",  "雨宿り",
      "あけぼの", "あんこ", "あられ", "アメンボ", "青虫", "暖かさ", "あかつき", "浴びる", "握手", "あはは",
      "安全", "あひる", "あさがお", "飽きない",  "朝イチ", "新たに", "悪臭", "あまから",
      "案内", "預かる", "遊び場", "危ない", "足あと", "あぶらあげ", "赤ん坊", "アキバ", "赤羽", "飽きる",
      "あちこち", "愛妻家", "明らか", "欺く", "あたふた", "空き缶", "争い", "アクセス", "あらあら", "朝寝坊",
      "青葉", "明るい", "与える", "悪霊", "案外", "呆れる",  "合わせる", "表す", "脂身",
      "天の川", "朝ぼらけ", "慌てる", "曖昧", "鮮やか", "あくまで", "足首", "雨雪", "あれこれ",
      "安心", "暗雲", "扱い", "あぐら", "雨具", "あごひげ", "アライグマ", "赤恥", "あぜ道", "赤の他人",
      "浅緑", "熱々", "唖然", "足風", "あっかんべー", "暴れる",  "あざらし", "荒波", "あぱあぱ",
      "足止め", "赤み", "明るさ", "暴れん坊", "甘口", "合わせもの", "あざ笑う", "天邪鬼", "仰ぐ",
          "あぶらぜみ", "あらし", "あげは", "あだな", "あごひも", "あきうど", "あらもの", "あおもの", "あかぬけ", "あじさい",
    "あげまき", "あかめ", "あつあげ", "あきんど", "あからさま", "あしおと", "あめあがり", "あせかき", "あかぐつ", "あさはか",
    "あひるぐち", "あたり", "あしなみ", "あぜち", "あぶく", "あらかじめ", "あまずっぱい", "あきひ", "あがた", "あみど",
    "青大将", "あかり", "あぶらげ", "あぶなげ", "あわただしさ", "あくび", "あかりとり", "あらかた", "あぶらそ", "あおぐ",
    "あまえんぼ", "あじわい", "あまのはら", "あだばな", "あだち", "あずまや", "悪習", "あかげ", "あひるなべ", "遊び心",
    "悪事", "赤魚金", "悪夢", "荒手前", "甘える", "甘党", "新しい", "甘夏", "預かり", "雨風",
    "アニメ", "網棚", "秋田こまち", "朝飯", "味噌汁", "青い", "アルパカ", "アルト", "味刺し", "赤色", "色",
    "犬", "椅子", "家", "石", "痛い", "妹", "苺", "命", "いただき", "池", "糸", "田舎", "稲荷",
    "入れ物", "芋掘り", "癒し", "いくら", "痛み", "いびき", "怒り", "祈り", "一葉", "いちじく", "いかだ",
    "色鉛筆", "犬の散歩", "いたずら", "一心不乱", "一日", "忙しい", "医者", "生け花", "一字", "一輪",
    "怒り", "鰯", "岩男", "一二三", "色合い", "入り口", "息かえり", "居眠り", "入れ墨", "稲妻", "色紙",
    "嫌がらせ", "生き物", "一日", "稲穂", "印刷", "一番", "一般的", "一大", "石段", "一番戦士", "一生",
    "今まで", "一作実", "異風", "一杯", "一部", "一緒", "移動", "一字", "一声", "一家", "いくつ",
    "一致", "いいわけ", "一段", "一鉄", "一面", "一円", "いくぶん", "一方", "一遍", "岩井", "一重",
    "一強", "異常", "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様", "異状", "家で",
    "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧",
    "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "入子", "磯尾",
    "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部", "印章", "一念", "一面",
    "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り", "医療", "力", "いずれ",
    "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬",
    "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪",
    "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部",
    "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り",
    "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前",
    "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク",
    "いけない", "印子", "猪", "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生",
    "一心", "一文", "一部", "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強",
    "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存",
    "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩",
    "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "入子", "磯尾", "一生懸命", "石杭",
    "一路", "入道雲", "一生二生", "一心", "一文", "一部", "印章", "一念", "一面", "一方", "一遍",
    "岩井", "異常", "一重", "一強", "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様",
    "異状", "家で", "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部",
    "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "入子",
    "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部", "印章", "一念",
    "一面", "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り", "医療", "力",
    "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸",
    "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子",
    "猪", "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文",
    "一部", "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく",
    "残り", "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象",
    "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい",
    "いちごミルク", "いけない", "印子", "猪", "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲",
    "一生二生", "一心", "一文", "一部", "印章", "一念", "一面", "一方", "一遍", "岩井", "異常",
    "一重", "一強", "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様", "異状", "家で",
    "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧",
    "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "入子", "磯尾",
    "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部", "印章", "一念", "一面",
    "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り", "医療", "力", "いずれ",
    "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬",
    "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪",
    "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部",
    "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り",
    "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前",
    "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク",
    "いけない", "印子", "猪", "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生",
    "一心", "一文", "一部", "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強",
    "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存",
    "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩",
    "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "入子", "磯尾", "一生懸命", "石杭", "一路",
    "入道雲", "一生二生", "一心", "一文", "一部", "印章", "一念", "一面", "一方", "一遍", "岩井",
    "異常", "一重", "一強", "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様", "異状",
    "家で", "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象",
    "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "入子", "磯尾",
    "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部", "印章", "一念", "一面",
    "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り", "医療", "力", "いずれ",
    "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前", "異物", "一寸", "一瞬",
    "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク", "いけない", "印子", "猪",
    "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生", "一心", "一文", "一部",
    "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強", "一応", "いただく", "残り",
    "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存", "一矢", "印象", "一面前",
    "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩", "いらっしゃい", "いちごミルク",
    "いけない", "印子", "猪", "色水", "入子", "磯尾", "一生懸命", "石杭", "一路", "入道雲", "一生二生",
    "一心", "一文", "一部", "印章", "一念", "一面", "一方", "一遍", "岩井", "異常", "一重", "一強",
    "一応", "いただく", "残り", "医療", "力", "いずれ", "戦争", "一様", "異状", "家で", "一読", "依存",
    "一矢", "印象", "一面前", "異物", "一寸", "一瞬", "一周", "一部", "印象", "一覧", "一白", "色彩",
    "いらっしゃい", "いちごミルク", "いけない", "印子", "猪", "色水", "いりこ", "いそしむ", "いしずえ", "いがく", "いりみち", "いりふね", "いちじくにゅうとう", "いしぶみ", "いきぬき",
      "いいん", "いちにちじゅう", "いってい", "いごこち","委嘱", "一人前", "歪", "一層", "いきなり", "色違い",
"いわば", "イライラ", "一応", "一斉に", "謂れ", "いやいや", "以前",
"イカサマ", "今までどおり", "いけない", "威嚇", "引力", "いい具合",
"生贄", "妹分", "一刻", "いざこざ", "いたちごっこ", "行き詰まり",
"怒り", "一等賞", "一瞬", "一種", "一部始終", "今更", "一元",
"板伝", "一戸建て", "一切", "礎", "言いなり", "潔い",
"石垣", "入れ替え", "一文字", "硫黄島", "石川", "いささか",
"一億", "一周", "色遣い", "いわゆる", "言わない", "イラスト", "一落",
"飲料", "井戸端", "一角", "一概に", "板橋区", "一般人",
"筏打ち", "一大事", "一月", "勇み足", "一足飛び", "今どき", "一言",
"一服",
"悪戯口", "生き地", "忌まわしい", "異文化", "遺伝子", "井の口", "今に", "イボ", "いいとこ取り", "息吹",
"イタコ", "一貫", "海", "歌", "腕", "兎", "梅", "牛", "宇宙", "うどん", "器", "生まれ",
"動き", "裏", "嘘", "鰻", "噂", "後ろ", "浮き輪", "浮世", "打ち明け", "鶯",
"歌声", "運動", "占い", "美しい", "羨ましい", "浮気", "受け取る", "馬", "丑寅",
"渦潮", "魚座", "うなじ", "歌う", "運転", "蛆虫", "海辺", "上下", "恨み", "魚市場",
"植木", "潤い", "宇宙船", "うっとり", "薄い", "受け入れ", "伺う", "うちわ", "腕前", "うるさい",
"失う", "浮き袋", "裏切り", "麗らか", "羨む", "奪う", "売り場", "埋め立てる", "旨み", "薄手",
"頷く", "動かす", "漆", "受け付ける", "上向き", "生み出す", "受け身", "売り切れ", "打ち放題", "裏付け",
"唸る", "承る", "浮かべる", "売り物", "疑う", "薄木", "打ち入り", "写し", "うるさ型", "裏成り",
"薄れる", "移ろう", "歌います", "上澄み", "腕時計", "上の空", "嬉し涙", "牛蛙", "請け合う", "うるち",
"俯す", "うぐう", "泡沫", "うねり", "裏表", "受け引き", "後ろ姿", "譫言", "うない人", "鬱陶しい",
"上回る", "うつぶせ", "売り上げ", "空け", "歌詠み", "海鳴り", "売り買い", "初々しい", "浮き沈み", "噂話",
"上顎", "産毛", "鶯餅", "裏門", "売れ行き", "自惚れ", "上塗り", "上皮", "売れ残り", "薄氷",
"上着", "裏漉し", "移り変わり", "項垂れる", "薄暗い", "浮かれる", "鬱憤", "嬉しがる", "うつげ", "蠢く",
"薄紙", "受け答える", "歌い手", "打ち合わせ", "浮い天気", "丑三つ時", "薄絹", "空ろ", "うがい", "憂い",
"唸く", "打ち掛ける", "うねる", "腕組み", "浮気の空", "伺い", "歌会", "産み", "雨氷", "写し写され",
"移り住み", "薄桃", "打ち寄せる", "移り気", "憂いの縁", "潮", "上葉", "薄汚れ", "浮き足", "裏取り",
"打ち出す", "初々しさ", "旨すぎる", "裏口", "裏文字", "嘘泣き", "空木", "うつ休み", "魚偏", "嬉しさ",
"上物", "産着", "鶯盆", "産土", "写し良い", "牛津", "鰻飯", "魚籠", "海の家", "打ち込み",
"姥", "上野", "薄黒", "移ろ", "嬉しがらせ", "産毛イボ", "失われる", "恨む", "浮かれすぎる", "裏声",
"浮いてくる", "現", "魚見", "海原", "薄味", "駅", "絵本", "鉛筆", "枝", "映画",
"笑顔", "英語", "選ぶ", "餌", "絵の具"
"液体", "江ノ島", "襟巻き", "海老", "描く", "円盤", "衛星", "英雄", "襟足", "エアコン",
"絵の具箱", "駅近", "遠足", "縁随", "枝豆", "縁側", "英文", "映像", "えこひいき", "園芸",
"宴会", "婉曲", "英断", "会釈", "枝切り", "駅員", "永住", "煙突", "演劇", "栄養",
"駅費", "えんどう", "偉そう", "縁付き", "江戸時代", "塩分", "閲覧", "沿岸", "英国", "縁起",
"延長", "鉛筆削り", "絵巻", "衣文", "駅中", "衛生員", "偉ぶん", "縁日", "縁紙", "堰堤",
"円高", "艶大な", "縁勝ち", "笑み", "営務", "遠近", "塩素", "永着", "餌入れ",
"絵解き", "演説", "演じる", "演出", "永久", "円熟", "円滑", "円心", "援助", "炎症",
"英文学", "縁談", "意固地",  "英知", "液化", "沿革", "円楽", "縁切り", "榎",
"衛生", "映写", "遠距離", "遠方", "演曲", "演技", "演舞", "宴然", "円形",
"演習", "艶色", "営業者", "演出家", "園芸家", "演技師", "音", "お菓子", "音楽", "お風呂", "お金", "お寺", "おやつ", "お店", "お箸", "お日様",
"お餅", "お花", "お父さん", "お母さん", "おじいさん", "おばあさん", "おもちゃ", "お弁当", "おかず", "折り紙",
"落とし物", "おしゃれ", "おにぎり", "お酒", "重い", "大きい", "美味しい", "面白い", "遅い", "大人",
"怒る", "踊る", "教える", "覚える", "終わる", "置く", "押す", "起こす", "降りる", "行う",
"思う", "恐れる", "怠る", "収まる", "抑える", "落ちる", "驚く", "送る", "思わず",
"おもてなし", "お祭り", "おせち", "おみくじ", "お土産", "お辞儀", "お任せ", "思い出", "お巡り",
"お宅", "お便り", "お手伝い", "お年玉", "踊り場", "お並び", "お話", "お参り", "趣", "親子",
"お休み", "お礼", "お詫び", "温泉", "温度", "応援", "王様", "王子", "横断", "往復",
"お陰", "小川", "掟", "屋上", "贈り物", "遅れ", "行い", "幼い", "惜しむ", "お互い",
"穏やか", "落ち葉", "夫", "男", "乙女", "脅し", "大人しい", "驚き", "お腹", "同じ",
"鬼ごっこ", "各々", "お早う", "帯", "お盆", "おまけ", "お守り", "お結び", "おめでとう", "面影",
"重さ", "表", "主に", "重り", "親子丼", "親父", "親指", "泳ぐ", "及ぼす", "織物",
"オルガン", "オレンジ", "卸し", "終わり", "恩返し", "恩恵", "温室", "温暖", "恩寵", "音符",
"お相", "追いかける", "追い越す", "生い茂る", "お出で", "追い抜く", "俺ら", "応じる", "応接", "応答",
"黄檗", "応募", "応用", "終える", "多い", "覆う", "大げさ", "大声", "大勢", "大空",
"概ね", "大物", "お帰り", "犯す", "女将", "拝む", "起き上がる", "置き換える", "置き去り", "置き時計",
"補う", "起きる", "奥様", "憶測", "奥地", "噯気", "送り仮名", "お稽古",
"厳か", "お小遣い", "収める", "惜しい", "おじいちゃん", "押し付け", "押し込む", "おじさん", "押し出す", "押し付ける",
"鴛鴦", "お釈迦", "お邪魔", "お正月", "押し寄せる", "白粉", "お勧め", "お世辞", "お世話", "襲う",
"恐らく", "恐れ", "恐ろしい"],
    "か行": [
"傘", "鞄", "川", "風", "顔", "紙", "肩", "亀", "角", "鍵",
"蛙", "兜", "鏡", "飾り", "カツオ", "カレイ", "カジキ", "火山", "河童", "隠れんぼ",
"海藻", "雷", "海岸", "考える", "鴎", "家族", "漢字", "火事", "格闘", "形",
"階段", "会社", "買い物", "関係", "観察", "家庭", "翡翠", "カチカチ", "カルタ", "可能",
"火曜日", "影", "鎌倉", "刀", "格好", "瓦", "果汁", "霞", "鍛冶屋", "乾杯",
"観光", "科学", "神様", "片仮名", "感謝", "感想", "硬い", "簡単", "可愛い", "悲しい",
"軽い", "格好いい", "かなり", "方々", "絵画", "悲しみ", "家畜", "快適", "株式", "韓国",
"女房", "火事の", "会社員", "蒲鉾", "かけっこ", "甲板", "感じる", "かりゆし", "隠れる",
"隠す", "傾く", "科目", "冠", "被る", "書き留める", "架け橋", "賢い", "歌謡曲", "影法師",
"歓迎", "関節", "官僚", "カップ", "海水", "垣根", "完成", "仮想", "掛ける",


"狐", "金庫", "教室", "着物", "切手", "聞く", "茸", "曲", "絹", "気合い",
"金魚", "汽車", "傷", "北風", "喫茶店", "切符", "救急車", "金曜日", "金元玉",
"傷口", "巨大", "季節", "啄木鳥", "恐竜", "麒麟", "機械", "希望", "規則", "機関車",
"金額", "競争", "汚い", "岸", "拒否", "近所", "弓道", "記録", "決まり", "緊張",
"記憶", "許可", "着替える", "記事", "強行", "近日", "兄弟", "黄色", "帰宅", "期間",
"切っ掛け", "郷土", "妃", "急行", "距離", "菊釘", "決める", "絆", "機嫌", "絹被ぎ",
"強調", "給食", "規範", "勤勉", "気候", "金属", "企業", "綺麗",
"極めて", "キチンと", "厳しい", "気難しい", "危険", "器具", "気を付ける", "寄贈", "休憩", "築く",
"傷付く", "兆し", "気持ち", "金銭", "来る", "キビキビ", "勤務", "気弱", "記号",
"既存", "金色", "気化学", "九州", "興味", "胡瓜", "肝試し", "球根", "金髪", "生糸",

"熊", "車", "薬", "口", "靴", "草", "雲", "首", "鯨", "果物",
"国", "空気", "窪み", "海月", "茎", "胡桃", "黒", "空港", "籤", "楠",
"くす玉", "籤引き", "唇", "繰り返し", "悔しい", "空間", "クレヨン", "鎖", "癖", "くしゃみ",
"空白", "加える", "組む", "崩れる", "繰り上げる", "釘", "詳しい", "クリーム", "草むら",
"草刈り", "暮らす", "空腹", "クリスマス", "クリスタル", "クリニック", "クローン", "クリーナー",
"クッキー", "熊本", "クイーン", "クラシック", "踵", "栗きんとん", "クッキリ", "首輪",
"砕ける", "暗闇", "くノ一", "腐る", "クタクタ", "桑原", "砕く", "苦労", "クエスト", "クリームパン",
"くびれ", "熊啄木鳥", "クイズ", "空想", "口癖", "挫ける", "擽る", "クルー", "区切り",
"潜る", "擽ったい", "草花", "九重", "暗い", "下る", "悔やむ", "窪む", "口籠もる",
"燻製", "繰り上がる", "クラブ", "口説く", "くどい", "嘴", "苦しむ", "桑",

            "煙", "警察", "結婚", "怪我", "消しゴム", "系統", "稽古", "ケリ", "桁", "計算",
"獣", "喧嘩", "建築", "毛虫", "蹴落とす", "見物", "鍵盤", "景色", "蹴散らす", "結果",
"血圧", "血液", "研修", "研究", "今朝", "経験", "契約", "経費", "掲示板", "健康",
"欠車", "賢者", "化粧", "結論", "検閲", "決定", "権", "県庁", "剣道", "建設",
"計画", "計算機", "欠点", "警告", "健全", "権利", "系譜", "経営", "憲法", "蛍光灯",
"権限", "痙攣", "警棒", "化粧品", "拳銃", "警戒", "軽蔑", "決心", "計算式", "蛍光ペン",
"権威", "警官", "掲示", "検範囲", "倹約", "傾斜", "警文", "化粧室", "決議", "警察署",
"健康診断", "欠席", "剣術", "欠如", "見解", "経験者", "結成", "快楽", "血縁", "経歴",
"刑務所", "結晶", "警視動車", "警備", "境内", "研究室", "蹴込む", "形成", "警視", "健診",
"軽傷", "系列", "検出", "敬意",
"子供", "氷", "公園", "声", "米", "焦げ", "耕地", "苔", "心", "拳",
"交通", "焦げ茶", "炬燵", "小道", "国語", "こけし", "航空", "小鳥", "粉雪", "困った",
"交差点", "交番", "試み", "行動", "交換", "工業", "航空機", "小雨", "駒",
"腰掛け", "行進", "胡椒", "拵える", "コスモス", "答える", "紅茶", "乞食", "こっち", "小遣い",
"交際", "甑", "婚約", "今晩", "効果", "皇居", "高原", "言葉", "小麦粉",
"構成", "効力", "鉱物", "肯定", "高速", "柿葺", "子犬", "子狐", "古方",
"講義", "公式", "国際", "航空便", "鯉のぼり", "工場", "公害", "混合", "蝙蝠",
"購買", "校舎", "高級", "個別", "後輩", "交付", "行津", "小包",
"こんにちは", "こんばんは", "こちら", "今後", "混と", "国民", "梱包", "堪える",
"根気", "懲りる", "黒板", "小槌", "国士", "高校", "告白", "困る", "言者"],
    "さ行": [
"桜", "塩", "寿司", "世界", "空", "猿", "新聞", "西瓜", "先生", "掃除機",
"魚", "仕事", "雀", "石鹸", "速達", "砂糖", "薩摩芋", "作品", "散歩", "作文",
"侍", "財布", "算数", "最初", "最後", "五月雨", "砂漠", "寒さ", "探す", "里親",
"刺身", "寒い", "祭壇", "撮影", "作物", "サッカー", "酸素", "作成", "裁判", "最適",
"細胞", "再生", "才能", "さっき", "作業", "さっぱり", "最高", "最低", "桜桃", "削除",
"最近", "歳月", "採決", "催促", "幸い", "詐欺", "作者", "作図", "作戦", "昨年",
"昨夜", "桜井", "酒", "叫び", "鎖国", "指図", "差し出し", "差し支え", "差し伸べる", "差し引き",
"刺す", "さすが", "授かる", "誘う", "定める", "作家", "錯覚", "早急", "作曲", "颯爽",
"察知", "査定", "悟る", "鯖", "捌く", "寂しい", "錆びる", "差別", "作法", "サボテン",
"サボる", "覚ます",


"試合", "仕上げ", "幸せ", "飼育", "子音", "支援", "塩辛い", "栞", "司会", "資格",
"しかし", "仕方", "叱る", "四季", "色彩", "敷地", "支給", "仕組み", "死刑", "試験",
"刺激", "茂る", "思考", "施策", "視察", "指示", "詩人", "使者", "市場", "静か",
"雫", "姿勢", "施設", "下着", "支度", "仕立て", "失業", "躾", "しっかり", "質問",
"指導", "市内", "品物", "死ぬ", "支配", "支払い", "芝居", "しばらく", "師範", "指標",
"渋い", "私物", "司法", "資本", "仕舞う", "締まる", "市民", "氏名", "占める", "紙面",
"指紋", "社員", "社会", "屈む", "借法", "車庫", "写真", "写生", "社宅", "社長",
"借金", "しゃっくり", "車道", "しゃべる", "シャボン", "三味線", "斜面", "洒落", "週", "収穫",
"宗教", "集合", "就職", "修正", "集中", "終点", "収入", "周辺", "宿題", "手術",
"主人", "主体", "主張", "出場", "出発", "出版", "趣味", "主役", "主要",


"素足", "水泳", "水産", "水準", "水素", "水中", "水筒", "水分", "水平", "睡眠",
"水面", "水曜", "吸う", "数学", "数字", "数値", "崇拝", "末", "末っ子", "素顔",
"姿", "好き", "隙間", "過ぎる", "すく", "救う", "少ない", "すぐ", "優れる", "スケート",
"透ける", "凄い", "少し", "筋", "鈴", "濯ぐ", "進む", "裾", "スタジオ", "スタンド",
"ステーキ", "素敵", "捨てる", "砂", "素直", "素晴らしい", "スピード", "全て", "滑る", "スポーツ",
"スマート", "住まい", "墨", "菫", "住む", "相撲", "すやすや", "すらすら", "スリ", "鋭い",
"すれ違う", "座る", "寸法", "スーツ", "スーパー", "スープ", "スムーズ", "スール", "スアナー", "スイーツ",
"スイッチ", "水田", "水道", "炊飯", "水没", "推量", "スウィング", "スウェーデン", "スウェート", "スウェートシャツ",
"スウェートパンツ", "スウェーデン語", "スウェーデン人", "スウェーデン料理", "スウェーデン旅行", "スウェーデン文化", "スウェーデン音楽", "スウェーデン映画", "スウェーデン文学", "スウェーデン芸術",
"スウェーデン建築", "スウェーデンデザイン", "スウェーデン家具", "スウェーデン服", "スウェーデン食品", "スウェーデン酒",


"性", "性格", "生活", "世紀", "請求", "清潔", "制限", "成功", "政策", "生産",
"政治", "正式", "精神", "成人", "製造", "生存", "生態", "成長", "製鉄", "生徒",
"青年", "整備", "政府", "生物", "成分", "製法", "生命", "制約", "西洋", "整理",
"成立", "西暦", "正論", "咳", "石炭", "責任", "世間", "背筋", "世帯", "節",
"設備", "切望", "説明", "節約", "背中", "狭い", "せめて", "攻める", "台詞", "世話",
"線", "繊維", "先鋭", "戦果", "選挙", "宣言", "専攻", "繊細", "戦士", "選手",
"扇子", "戦争", "選択", "船長", "選定", "宣伝", "仙人", "先輩", "扇風機", "選別",
"洗面", "専門", "川柳", "先例", "セーター", "セール", "セーフティー", "セーラー", "背油", "制圧",
"星雲", "声援", "西欧", "成果", "誓願", "正教", "整形", "制裁", "静止", "性質",
"青春", "聖書", "清勝", "正常", "整数",


"そう", "相違", "憂鬱", "騒音", "創価", "総会", "総額", "送金", "遭遇", "送検",
"倉庫", "総合", "操作", "創作", "掃除", "葬式", "送信", "総数", "想像", "相続",
"早退", "相談", "装置", "想定", "相当", "遭難", "壮年", "相場", "総理", "添える",
"足", "即座", "促進", "即席", "測定", "速度", "速報", "側面", "測量", "底",
"祖国", "素材", "組織", "蘇生", "祖先", "注ぐ", "育てる", "措置", "そっくり", "そっと",
"袖", "外", "備える", "そのうち", "そのまま", "蕎麦", "祖父", "祖母", "粗末", "背く",
"染める", "そよ風", "それ", "それぞれ", "揃う", "揃える", "損", "損害", "尊敬", "存在",
"損失", "尊重", "ソース", "ソーセージ", "ソーダ", "ソープ", "粗悪", "そいつ", "草案", "創院",
"総円", "相応", "創刊", "早期", "総計", "創傑", "草原", "走行"] ,
    "た行": [
"太陽", "地球", "机", "手紙", "時計", "食べ物", "地図", "翼", "天気", "鳥",
"タンポポ", "地下鉄", "梅雨", "手袋", "豆腐", "卵", "蝶々", "津波", "天ぷら", "灯台",
"建物", "貯金箱", "積木", "天国", "トウモロコシ", "たこ焼き", "塵取り", "土筆", "天井", "扉",
"台風", "提灯", "佃煮", "天皇", "峠",
"畳", "調味料", "つくね", "天風", "唐黍", "太鼓", "長男", "作り物", "転覆", "唐辛子",
"狸", "冬至", "橘", "挑戦", "作り事", "東風", "彫刻", "冬季", "朝食",
"作り方", "投票", "玉葱", "聴力", "作り替え", "当選", "竹馬", "調和", "作り出す", "灯心",
"調査", "作り上げる", "遂に", "竹蜻蛉", "頂点", "作り出し", "透明", "調節", "尊い", "竹藪",
"調子", "闘将",


"力", "小さい", "近い", "違う", "丁度", "ちょっと", "ちゃんと", "注意", "挑戦", "蝶々",
"塵取り", "貯金", "注文", "調子", "近づく", "頂戴", "注射", "知恵", "彫刻", "長男",
"中心", "長命", "調和", "調整", "注目", "調理", "頂点", "長女", "聴衆",
"長所", "調節", "中立", "調査", "調停", "長期", "兆候", "聴診", "朝夕",


"机", "翼", "梅雨", "積木", "土筆", "佃煮", "つくね", "作り物", "作り話", "作り事",
"作り方", "作り替え", "作り出す", "作り上げる", "作り出し", "作る", "使う", "疲れる", "続く",
"伝える", "包む", "繋ぐ", "呟く", "潰す", "摘む",
"積もる", "強い", "辛い", "ツルツル", "連れる", "ツンツン", "蕾", "瞑る", "尽きる", "付き合う",
"突き抜ける", "突き当たる", "突き止める", "突き進む",


"手紙", "手袋", "天ぷら", "天国", "天井", "天皇", "天風", "転覆", "天気", "手伝う",
"手洗い", "手当て", "手編み", "丁寧", "手掛ける", "適当", "手強い", "手探り", "手品", "手摺",
"てっきり", "徹夜", "手抜き", "手放す", "手控え", "手前", "手回し", "照らす", "照れる", "手分け",
"展開", "点検", "転校", "天才", "天使", "点数", "天体", "てんでん", "転倒", "天然",


"時計", "鳥", "豆腐", "灯台", "トウモロコシ", "扉", "灯籠", "峠", "唐黍", "唐辛子",
"冬至", "東風", "冬季", "投票", "当選", "灯心", "遂に", "透明", "尊い", "闘将",
"遠い", "通す", "尖る", "解く", "得意", "特別", "溶ける", "閉じる", "戸棚", "途中",
"突然", "届く", "届ける", "留まる", "飛ぶ", "止まる", "止める", "友達", "捉える", "取り敢えず"],
    "な行": [
],
    "は行": [
"花", "人", "船", "ヘビ", "星", "橋", "ひまわり", "冬", "部屋", "本",
"ハサミ", "飛行機", "風船", "へそ", "ほうき", "畑", "羊", "袋", "平和", "ホタル",
"拍手", "ひげ", "布団", "返事", "骨太", "はがき", "ひなまつり", "風呂敷", "変化", "包丁",
"はやぶさ", "ひのき", "ふきん", "へんてこ", "ほうれんそう", "花火", "ひよこ", "ふくらはぎ", "へそくり", "ほっぺた",
"はんこ", "ひらがな", "ふくろう", "へそまがり", "宝石", "花見", "瞳", "吹き出し", "へそくりがま", "ほうずき",
"半分", "独り言", "覆面", "へそくりがめ", "包丁人", "花嫁", "一人ぼっち", "複雑", "ほうれんそうごはん",
"はなみず", "ひとりじめ", "ふくらむ", "へそくりばこ", "ほうれんそうサラダ", "はなたれ", "ひとりぼっち", "ふくらはぎ", "へそくりがめ", "ほうれんそうのおひたし",
],
"ま行": [
    "舞", "舞う", "前", "巻き", "撒く", "マグ", "負け", "曲げ", "孫", "増し",
    "増す", "まず", "マゾ", "また", "まだ", "町", "松", "的", "窓", "まな",
    "真似", "間の", "麻痺", "まぶ", "魔法", "まま", "まみ", "豆", "まら", "毬",
    "丸", "稀", "魔王", "魔界", "紛い", "真顔", "籬", "任す",
    "真鴨", "曲がり", "蒔絵", "牧場", "紛れ", "間際", "馬草", "間口",
    "枕", "鮪", "負ける",

    "味方", "神輿", "岬", "見せる", "霙", "みたい", "見出し", "満ちる", "緑", "港",
    "実る", "見張る", "身振り", "見本", "見舞う", "味覚", "三日月", "御門", "磨く", "見極める",
    "見込む", "見定める", "みすぼらしい", "見捨てる", "湖", "自ら", "水着", "見せびらかす", "味噌汁", "満たす",
    "見つかる", "見つける", "見つめる", "認める", "見なす", "源", "見習う", "見逃す", "蓑虫", "見計らう",
    "身分", "見守る", "見回り", "土産", "都", "未来", "魅力", "見分ける", "見渡す", "皆",

    "六日", "向かい", "向かう", "迎え", "昔", "麦茶", "向ける", "無限", "虫歯", "矛盾",
    "息子", "結び", "結ぶ", "娘", "無駄な", "夢中", "空しい", "紫", "無料", "群れる",
    "無一物", "無一文", "無益", "無縁", "無音", "無害", "無学", "向かい合う", "向かい風", "迎え撃つ",
    "無関係", "無関心", "剥き出し", "むきになる", "向き物", "無休", "無力", "無菌", "報い", "むくむ",
    "向こう", "酷い", "無骨", "むさぼる", "無残", "蒸し暑い", "無視する", "無実", "狢", "無慈悲",

    "迷惑", "目上", "恵む", "巡る", "目指す", "目下", "珍しい", "目立つ", "めちゃくちゃ", "メッキ",
    "めでたい", "目の前", "芽生え", "目まい", "メモリ", "明暗", "明快", "名曲", "名作", "名刺",
    "命じる", "瞑想", "名物", "名誉", "命令", "迷路", "雌牛", "目移り", "夫婦", "目方",
    "眼鏡", "恵まれる", "恵み", "巡り合う", "巡り合わせ", "目配せ", "目配り", "盲", "目黒", "目敏い",
    "目覚まし", "召し上がる", "召し抱える", "召使い", "雌しべ", "目印", "めだか", "目玉", "めちゃめちゃ", "滅多",

    "儲ける", "申す", "燃える", "潜る", "もしも", "もたれる", "勿論", "もっと", "もてなす", "戻す",
    "物差し", "もはや", "もめる", "燃やす", "模様", "詣でる", "毛布", "萌黄", "もがく", "もぎ取る",
    "目撃", "木材", "目次", "目的", "目標", "もぐさ", "もぐもぐ", "模型", "模索", "文字",
    "若しくは", "もたらす", "餅", "持ち上げる", "持ち切り", "持ち越し", "持ち出す", "持ち直す", "持ち主", "持ち場",
    "持つ", "もつれる", "もてあそぶ", "モテる", "求める", "もども", "物語", "物心", "物事", "物好き"
],

"や行": [
    "八重", "焼き", "山羊", "役", "やぐ", "焼け", "椰子", "野次", "痩せ", "やち",
    "奴", "やと", "宿", "屋根", "やひ", "藪", "野暮", "山", "闇", "止む",
    "いや", "揶揄", "槍", "やる", "柔", "やあい", "やあど", "刃", "やいん", "八重",
    "八重歯", "野縁", "八百屋", "夜会", "野外", "館", "やがて", "やから", "薬缶", "焼絵",

    "勇気", "郵便", "有名", "浴衣", "雪解け", "湯気", "漱ぐ", "譲る", "委ねる", "ゆっくり",
    "茹でる", "余裕", "輸入", "指差す", "夢見る", "湯浴み", "唯一", "遺言", "由緒", "憂鬱",
    "遊園地", "夕方", "勇敢", "夕暮れ", "幽谷", "優秀", "友人", "郵送", "誘導", "夕日",
    "昨夜", "夕焼け", "幽霊", "故に", "愉快", "縁", "歪む", "行き交う", "雪国", "雪だるま",

    "よづり", "予定", "与党", "淀み", "夜中", "夜長", "夜泣き", "夜鍋", "余熱", "余念",
    "よのぎ", "余輩", "余白", "夜明け", "よいしょ", "宵", "宵止め", "夜風", "予感", "預金",
    "翌朝", "翌月", "翌日", "抑制", "欲望", "余計", "横切る", "寄越す", "汚れ", "予算",
    "善し悪し", "予想", "予測", "余所見", "余所者", "よだれ", "よちよち", "四つ角", "四つ葉", "予定",
    "夜通し", "夜な夜な", "夜逃げ", "余熱", "余白", "呼びかける", "呼び出す", "呼び止める", "余分", "予報"
],

"ら行": [
    "来", "楽", "らし", "乱", "来年", "落第", "楽園", "乱暴", "来月", "落語",

    "利口", "理想", "理由", "料理", "旅館", "旅行", "林檎", "臨時", "力む", "理屈",
    "理性", "利息", "利得", "竜", "了解", "理解", "利害", "力作", "力説", "力点",
    "陸軍", "陸地", "陸路", "理系", "離婚", "離散", "利子", "利潤", "理数", "理想郷",
    "離体", "離脱", "律儀", "立教", "立派", "利点", "利得", "入院", "理髪", "リビング",

    "類型", "類似", "留守番", "流転", "瑠璃色", "流浪", "類人", "累代", "累積", "類販",
    "累々", "ルサンチマン", "ルメン", "瑠璃瓦", "瑠璃珠職", "瑠璃雀", "瑠璃鶲", "瑠璃祭り", "ルワンダ", "るんるん",

    "冷蔵", "冷静", "歴史", "恋愛", "連続", "連絡", "列車", "歴代", "蓮根", "連係",
    "冷房", "例外", "歴史家", "連結", "連盟", "霊気", "礼儀", "冷血", "冷酷", "零細",
    "礼節", "例題", "冷凍", "礼拝", "黎明", "令和", "歴戦", "歴代", "歴任", "歴法",

    "蝋燭", "録音", "論理", "朗読", "廊下", "路上", "論文", "老人", "労働", "六角",
    "録画", "露店", "牢獄", "労費", "論争", "朗詠", "廊下", "老眼", "老朽", "労作",
    "老女", "老衰", "蝋燭", "朗読", "浪人", "老婆", "老耄", "録画", "ろくでなし", "露骨"
],

"わ行": [
    "若", "訳", "和語", "技", "鷲", "綿", "鰐", "わび", "藁", "悪",
    "湾", "若者", "別れ", "惑星", "山葵", "私", "輪投げ", "忘れる", "笑い", "割合",
    "猥褻", "わいわい", "若い", "若返る", "若木", "若草", "若さ", "沸かす", "分かち", "分かち合う",
    "分つ", "若手", "若葉", "若水", "若芽", "若やぐ", "分かれ道", "分かれ目", "沸き上がる", "和気藹々",
    "沸き返る", "湧き出す", "湧き出る", "弁える", "脇道", "脇目", "脇役", "枠", "枠組み", "惑星"
]



}

# 五十音の段ごとの単語グループ - サンプルデータのみ
COLUMN_WORDS = {
  "あ段": [
    "青空", "顎", "赤", "朝", "足", "頭", "汗", "雨", "明かり", "挨拶",
    "顎", "甘い", "甘み", "浅い", "彩る", "朝日", "朝顔", "暁", "赤鼻", "朝霧",
    "歩く", "歩み", "洗う", "浴びる", "穴", "油", "雨雲", "雨降り", "飴", "網",
    "雨上がり", "荒い", "粗い", "嵐", "表す", "改める", "新しい", "争う", "危ない", "アヒル",
    "案内", "青梅", "青虫", "赤ちゃん", "安全", "商い", "遊び", "空き缶",
    "会う", "鯵", "蚊", "科学", "影", "鏡", "家具", "学校", "風", "金",
    "髪", "紙", "体", "絡む", "数", "壁", "神", "鐘", "上手", "構う",
    "考える", "火事", "梶", "歌う", "数える", "勝つ", "飾る", "花瓶", "株", "噛む",
    "刈る", "借りる", "枯れる", "顔", "返す", "鞄", "被る", "川", "皮", "変わる",
    "交わる", "乾く", "咳", "駆ける", "欠ける", "陰", "囲む", "通う", "飼う", "香る",
    "刻む", "殻", "カステラ", "紙芝居", "髪切り", "刀", "鎌", "紙粘土",
    "サーカス", "財布", "魚", "桜", "探す", "刺す", "指す", "触る", "誘う", "猿",
    "皿", "去る", "騒ぐ", "盛り上がる", "覚ます", "冷ます", "冷やす", "白い", "探る", "叫ぶ",
    "避ける", "鮭", "酒", "先", "咲く", "裂く", "削る", "サザエ", "鯖", "サボテン",
    "座布団", "サラダ", "ざる", "三角", "酸っぱい", "サンタ", "三味線", "裁く", "栄える", "騒がしい",
    "鯖缶", "酒粕", "桜貝", "桜色", "散歩", "下がる", "砂場",
    "タクシー", "竹", "武士", "タコ", "助ける", "戦う", "楽しい", "頼む", "叩く", "立つ",
    "建つ", "辿る", "携える", "尋ねる", "足す", "出す", "黙る", "食べる", "溜まる", "溜める",
    "足りる", "黄昏る", "足る", "垂れる", "例える", "盾", "卵", "タバコ", "足袋", "旅",
    "タンス", "樽", "短歌", "俵", "平ら", "直る", "台", "互い", "谷", "種",
    "玉", "誰", "絶える", "戦い", "宝", "側", "谷間", "多数", "楽しむ", "田んぼ",
    "タンポポ", "竹馬", "竹とんぼ", "竹やぶ", "竹輪", "助かる",
    "七夕", "名前", "波", "奈良", "習う", "泣く", "鳴く", "眺める", "投げる", "撫でる",
    "舐める", "慣れる", "悩む", "並ぶ", "生る", "鳴り響く", "鍋", "涙", "梨", "ナマズ",
    "南京玉", "波打ち際", "菜の花", "生花", "名札", "なぞり", "ナメクジ", "縄", "鯰", "生肉",
    "ナルト", "成り立つ", "鳴子", "縄跳び", "七つ", "夏休み", "懐かしい", "名残り", "泣き声", "慰める",
    "無くす", "流す", "流れる", "名刺", "なすび", "夏祭り", "ナスカの地上絵",
    "花", "歯", "箱", "葉", "蜂", "鼻", "橋", "走る", "始まる", "離す",
    "弾く", "挟む", "挟まる", "話す", "放す", "離れる", "張る", "貼る", "晴れる", "払う",
    "腫れる", "掃く", "刃", "端", "歯医者", "灰", "蠅", "墓", "花火", "葉巻",
    "浜辺", "速い", "早い", "腹", "梅雨", "華やか", "鼻歌", "春風", "春雨", "鼻水",
    "半分", "花束", "花見", "花嫁", "腹黒い", "春休み", "針", "刃物", "場所", "端っこ",
    "晴れ間",
    "真ん中", "回る", "豆", "間に合う", "守る", "磨く", "待つ", "纏う", "祭り", "迷う",
    "舞う", "混ざる", "交わる", "負ける", "曲げる", "迷わす", "巻く", "蒔く", "撒く", "巻き込む",
    "紛れる", "任す", "招く", "学ぶ", "真似る", "妬む", "丸い", "回る", "真っ直ぐ", "間",
    "前", "真", "毎日", "毎朝", "毎晩", "毎回", "舞台", "マット", "的", "マフラー",
    "眉毛", "マッチ", "松", "祭", "魔法", "真っ白", "真夜中", "丸", "真珠", "マーブル",
    "マグロ", "マスク", "枕", "窓", "マヨネーズ", "真っ黒", "真っ赤", "婿", "幻", "豆腐",
    "窓枠", "松茸", "祭り囃子", "天気", "間食", "孫", "松林", "マラソン", "真実", "丸太",
    "マントヒヒ", "松原", "真昼",
    "山", "屋根", "病", "野菜", "役", "休む", "焼く", "火傷", "辞める", "破る",
    "行う", "優しい", "止む", "痩せる", "養う", "破れる", "病む", "止める", "和らぐ", "闇",
    "家", "山脈", "山岳", "野原", "八百屋", "役者", "山猫", "野球", "役所", "薬局",
    "屋敷", "安物", "山頂", "宿", "宿屋", "宿題", "宿泊", "歪み", "薬品", "薬指",
    "山林", "山道", "山火事", "屋台", "山小屋", "屋形船", "野原", "役立つ", "焼き鳥", "焼きそば",
    "山彦", "山百合", "野良猫", "野良犬", "闇夜", "闇市", "病気", "休み", "柔らかい", "易しい",
    "野菜", "山菜", "屋内", "屋外", "役割", "約束", "野望", "野生", "優雅", "夜間",
    "矢印", "野球場", "薬草", "役人", "夜食", "野次馬", "八つ", "山羊", "野鳥", "野点",
    "山登り", "薬湯",
    "楽", "来る", "雷", "落ちる", "楽しむ", "散らかる", "散らかす", "塗る", "ラジオ", "洗う",
    "ラッパ", "ランドセル", "ラッコ", "ラクダ", "落雷", "落葉", "落花", "落書き", "落ち葉", "落ち込む",
    "来客", "来店", "ラディッシュ", "ラムネ", "ランプ", "乱暴", "乱気流", "乱丁", "乱雑", "乱世",
    "ラフな", "らっきょう", "ラグビー", "ラケット", "ラップ", "ラベンダー", "ラベル", "拉致", "ラミネート", "ラリー",
    "落第", "ラウンド", "ランキング", "ラウンジ",
    "輪", "笑う", "割る", "分かる", "湧く", "忘れる", "割れる", "沸く", "藁", "鷲",
    "ワサビ", "綿", "話", "ワイン", "和風", "ワイド", "輪ゴム", "割引", "ワープ", "ワット",
    "ワナ", "早稲田", "渡り鳥", "ワックス", "ワッフル", "話題", "割り当て", "割り箸", "分かれ道", "分かりやすい",
    "分かち合う", "忘れ物", "分ける", "湧き水", "脇腹", "分からず屋", "割った", "和太鼓", "和紙", "和室",
    "和服", "和牛", "分からない", "ワラビ", "腕", "腕時計", "わらび餅", "笑顔", "ワルツ", "ワルシャワ"
],

"い段": [
    "意味", "位置", "一つ", "生き物", "息", "行く", "異国", "以下", "偉大", "医学",
    "いえ", "家", "イギリス", "異議", "イカ", "怒る", "イカサマ", "育つ", "石", "椅子",
    "異常", "医者", "急ぐ", "痛い", "悪戯", "糸", "一緒", "一時", "一本", "言う",
    "戦", "息子", "居間", "今", "いりこ", "彩", "犬", "命", "意見", "生きる",
    "イヤリング", "芋", "妹", "イルカ", "色", "印刷", "印象", "隠す", "飲む", "祈る",
    "イスラム", "入れる", "医療", "允許", "岩", "イワシ",
    "キノコ", "機械", "稀", "既に", "キツネ", "危険", "気体", "キロ", "議会", "議論",
    "着る", "切る", "切れる", "極める", "気候", "機関", "聞く", "効く", "利く", "菊",
    "きしむ", "北", "汚い", "気付く", "絹", "昨日", "キャベツ", "キャンプ", "急須", "休暇",
    "教会", "教科書", "胸", "協力", "巨大", "許可", "漁師", "金魚", "銀行", "禁止",
    "近所", "金庫", "金曜日",
    "品", "白い", "知る", "支持", "指示", "詩人", "資金", "時代", "下着", "仕度",
    "芝生", "渋い", "渋滞", "シミ", "紙幣", "写真", "社会", "謝罪", "車庫", "視覚",
    "持久力", "静か", "地震", "沈む", "シジミ", "シソ", "仕事", "敷布", "敷く", "しっぽ",
    "湿る", "萎びる", "師匠", "システム", "姿勢", "自然", "失敗", "質問", "実験", "シナリオ",
    "芯", "しのぶ", "支払い", "詩", "栓", "標識", "塩分", "幸せ", "しまう", "シマウマ",
    "島々", "染みる", "紙面", "市民", "締める", "閉める", "湿気", "占める", "示す", "霜",
    "下", "士", "シャツ", "写す", "借金", "市役所", "車輪", "社長", "砂漠", "シャベル",
    "シャンプー", "シューズ", "週間", "週末", "州", "獣", "収入", "渋滞", "収穫", "シュート",
    "シュークリーム", "趣味", "主役", "障害", "定規", "醤油", "証明", "証拠", "小学校", "消費",
    "少年", "焦点", "紹介", "商店", "将来", "少し", "少ない", "情報", "消化", "小説",
    "詳細", "シリーズ", "知り合い", "知人", "私立", "視力", "尻尾", "親切", "新聞", "神社",
    "真珠", "人口", "紳士", "進出", "信念", "診断", "心配", "深夜", "新年", "診察",
    "親戚", "心", "森林", "シンプル",
    "散歩", "知識", "枝", "七十", "シナモン", "シアン", "視線", "紫陽花", "七時", "四角",
    "辛子", "支店", "支度", "シチュー", "しとど", "雫", "仕立て", "支流", "しぼむ", "締切",
    "仕分け", "四方", "シーソー", "シーツ", "シール", "四季", "指揮", "式", "仕切り", "敷地",
    "しおらしい", "しおしお", "しなやか", "凌ぐ", "芝居", "芝刈り", "しぶとい", "渋面", "シフォン", "嗜好",
    "私情", "思索", "視察", "シスター", "子守唄", "仕草", "沈める", "シャッター", "謝意", "社交",
    "社交界", "しゃきしゃき", "詐欺", "邪気", "爵位", "灼熱", "試薬", "視野", "試用", "仕様",
    "試験", "試作", "私邸", "自慢", "しみじみ", "染み抜き", "清水", "志望", "しめやか", "締め切り",
    "占い師", "霜柱", "紙片", "屍", "資産", "持参", "事態", "下図", "シーン", "刺繍",
    "支給", "シジミ汁", "視聴率", "指定席", "自転車", "しどけない", "忍び足", "しのぐ", "詩吟", "紫煙",
    "枝垂れ", "子守", "四六時中", "思慮", "私用", "詩歌", "時価", "色紙", "しぐさ", "しこり",
    "四股", "思考", "視察", "至極", "死語", "枝豆", "霜降り", "指紋", "試問", "志向",
    "自己", "詞", "資質", "詩集", "視聴", "思想", "至上", "士気", "刺激", "史跡",
    "四天王", "使途", "至難", "始発", "指標", "脂肪", "姉妹", "市場", "死亡", "シベリア",
    "思案", "紫苑", "試行", "飼育", "資源", "持続", "時刻", "叱咤", "湿地", "支柱",
    "シーツ", "シール", "シート", "シェア", "姿", "地味", "染み", "染織", "氏名", "シミュレーション",

    "チーズ", "茶", "力", "違い", "血", "散らす", "縮む", "縮まる", "塵", "地図",
    "秩序", "窒息", "チェス", "茶色", "茶碗", "チャンス", "彫刻", "貯金", "調査", "調子",
    "調理", "蝶", "長所", "頂上", "長方形", "挑戦", "朝食", "超える", "聴覚", "朝刊",
    "著者", "貯蓄", "直角", "直線", "沈黙", "賃金", "沈殿", "珍しい", "沈没", "追加",
    "注目", "中心", "忠告", "駐車", "注意", "中学", "中間", "昼食", "中立", "中止",
    "注射", "柱", "抽象", "宙返り", "駐在", "抽出", "チューリップ", "懲罰", "長靴", "腸",
    "蝶々", "調味料", "長寿", "庁舎", "彫像", "蝶番", "庁", "跳ねる", "散る", "塵取り",
    "チーム", "地位", "チェーン", "血管", "契約", "知恵", "恥辱", "地球", "竹", "遅刻",
    "遅延", "稚魚", "チケット", "蓄積", "父親", "知能", "地平線", "地方", "茶道", "着陸",
    "着色", "着席", "着実", "着用", "昼寝", "注文", "注釈", "宙吊り", "頂く",
    "ニット", "煮る", "似てる", "逃げる", "握る", "鈍る", "荷物", "にやり", "匂い", "煮込む",
    "二階", "肉", "日記", "日曜日", "人間", "人形", "妊娠", "人生", "人類", "認める",
    "入院", "入社", "乳牛", "入場", "入浴", "尿", "日本", "日", "庭", "二回",
    "二十", "荷台", "煮物", "ニワトリ", "人参", "人気", "二十日", "二人", "日光", "日差し",
    "入金", "入口", "入学", "西", "錦", "虹", "ニヤニヤ", "ニキビ", "にきび", "ニンニク",
    "にこやか", "にらめっこ", "二万", "ニュース", "日常", "日時", "日課", "苦い", "鈍い", "憎い",
    "仁王", "庭園", "庭木", "俄か", "疾し", "二十歳", "二十万", "ニジマス", "二次会", "二度",
    "二重", "日照", "日数", "ニス", "日中", "二束三文", "ニセモノ", "煮炊き", "日当たり", "日中",
    "二年", "二年生", "匂う", "ニオイザクラ", "ニコチン", "ニコニコ", "憎しみ", "憎む", "鈍る", "逃す",
    "二男", "ニキビ面", "荷造り", "二枚舌", "二枚目", "ニモノ", "ニラ", "二流", "韮", "ニレ",
    "二連", "人込み", "任務", "認可", "人格", "認識", "任期", "人権", "妊娠中", "妊婦",
    "人目", "人並み", "任命", "認知", "妊娠中絶", "人脈", "認知症", "刃物", "庭番", "鶏鳴",
    "荷駄", "二束", "二束草鞋",
    "火", "陽", "灯", "光", "干す", "引く", "挽く", "弾く", "轢く", "惹く",
    "退く", "日々", "冷やす", "比べる", "響く", "光る", "引き出し", "蛙", "蘭", "姫",
    "皮膚", "秘密", "被害", "日付", "控え", "左", "羊", "広げる", "貧困", "広い",
    "広がる", "飛行", "悲劇", "飛行機", "飛行場", "比較", "悲観", "非常", "筆", "必死",
    "必要", "一言", "瞳", "人", "一時", "一家", "一つ", "一区", "一休み", "酷い",
    "一回", "一巻", "一見", "一時", "一途", "一塁", "引力", "響き", "ひねくれる", "陽炎",
    "雛人形", "紐", "皮肉", "批判", "日程", "一杯", "貧血", "品詞", "非難", "避難",
    "悲鳴", "秘める", "ヒマワリ", "紐", "百", "百科", "百万", "氷", "冷える", "ひれ",
    "疲労", "広場", "肥料", "昼間", "肥満", "紐", "紐解く", "扉", "ヒーター", "ヒーロー",
    "ヒット", "否定", "ビタミン", "美人", "美術", "びっくり", "必勝", "筆跡", "必然", "ヒツジ",
    "匹", "ビデオ", "ヒナ", "日向", "非武装", "皮膚", "ヒョウ", "俵", "秒", "標識",
    "標準", "表現", "表示", "病院", "病気", "評価", "表紙", "評論", "美容", "表面",
    "病室", "表情", "評判",
    "三つ", "認める", "見る", "診る", "実", "身", "味", "実る", "見える", "見つける",
    "満ちる", "晝", "道", "未満", "耳", "南", "実家", "ミカン", "右", "味噌",
    "短い", "店", "見せる", "水", "湖", "見通し", "密か", "蜜柑", "密接", "みっともない",
    "三日月", "見栄え", "ミニ", "三人", "醜い", "身震い", "見直す", "魅力", "見る見る", "未来",
    "ミルク", "見分け", "見渡す", "民族", "眠る",
    "霧", "巫女", "味覚", "三角", "ミキサー", "未熟", "見苦しい", "醜悪", "見込み", "見頃",
    "見事", "ミステリー", "三男", "見掛ける", "三日", "三年", "醜女", "三ツ星", "三日四日", "醜態",
    "三毛猫", "水鏡", "美紀子", "茎", "見切り", "ミクロ", "ミクロン", "身近", "見苦しい", "身ごもる",
    "魅惑", "岬", "みそか", "満ちる", "密集", "密植", "密接", "密談", "蜜月", "密閉",
    "見て取る", "未知", "道順", "身長", "満ちる", "身近", "未定", "緑色", "身動き", "実り",
    "三角形", "ミモザ", "脈", "脈拍", "都", "都道府県", "ミルフィーユ", "魅入られる", "見抜く", "醜い",
    "視る", "未開", "見当", "見当たる", "見聞き", "見慣れる", "見栄", "見栄っ張り", "ミツマタ", "密輸",
    "三ツ", "ミット", "見取り図", "見どころ", "ミーティング", "ミード", "ミーム", "身内", "見比べる", "見まがう",
    "ミュージアム", "ミリ", "未練", "魅了", "明滅", "見積もり", "視る", "澪標", "見目", "味方",
    "水浴び", "水着", "水族館", "水難", "水際", "三者三様", "店番", "水飴", "水彩", "水仙",
    "水辺", "水泳", "水玉", "店屋", "水道", "水筒", "水中", "店先", "水車", "水牛",
    "簑", "身震い", "未発表", "身幅", "身勝手", "実子", "身なり", "身の丈", "身分", "身繕い",
    "蓑虫", "身持ち", "身元", "実入り", "見落とし", "身代わり", "見破る", "実らす", "実る", "実り",
    "未練", "魅惑", "魅力", "見惚れる", "見る目",
    "理解", "利益", "利子", "リス", "リボン", "旅", "料金", "両親", "利用", "流行",
    "理由", "量", "漁", "リアル", "リード", "陸", "リサイクル", "リスク", "リスト", "リズム",
    "立方", "リビング", "リボルバー", "略す", "略奪", "流暢", "粒子", "リクエスト", "リッチ", "栗鼠",
    "略語", "流砂", "旅券", "緑茶", "旅情", "理性", "理想", "理論", "リハビリ", "リバース",
    "リポート", "略図", "龍", "流氷", "粒", "リング", "林業", "リストラ", "立体", "リニア",
    "リハーサル", "リモコン", "リヤカー", "リラックス", "利息", "リース", "リンカーン", "林檎", "リンス", "輪廻",
    "リーフ", "リーダー",
    "リカバリ", "リグラ", "理財", "理事", "利子", "リシャフト", "リスナ", "リスナー", "リセット", "理想",
    "リゾット", "リタイア", "リチウム", "律儀", "立脚", "立証", "リッター", "リットル", "リップ", "リハビリ",
    "リバイバル", "リファイン", "リフォーム", "リフレーミング", "リベンジ", "理法", "リボン", "略儀", "略記", "略言",
    "略語", "略式", "略図", "略述", "略奪", "リモート", "リヤカー", "理由", "流言", "流行り",
    "流産", "リューマチ", "竜巻", "流水", "リュック", "領域", "了解", "良好", "了承", "料金",
    "料理", "両極", "両側", "両者", "両手", "両日", "両方", "療法", "量産", "領主",
    "両陛下", "両目", "両用", "両翼", "旅客", "緑黄色", "緑青", "リレー", "リング", "輪唱",
    "リンス", "リンネル", "リンパ", "林立", "臨界", "臨時", "倫理", "輪郭", "リーグ", "リーズナブル",
    "リーダーシップ", "リーフレット", "履歴", "理念", "利根", "リズミカル", "リスキー", "リセット", "理知的", "リバティ"
],

"う段": ["兎", "海", "歌", "腕", "梅", "牛", "宇宙", "うどん", "器", "生まれ", "動き", "裏", "嘘", "鰻", "噂", "後ろ", "浮き輪", "浮世", "打ち明け", "鶯", "歌声", "運動", "占い", "美しい", "羨ましい", "浮気", "受け取る", "馬", "丑寅", "渦潮", "魚座", "うなじ", "歌う", "運転", "蛆虫", "海辺", "上下", "恨み", "魚市場", "植木", "潤い", "宇宙船", "うっとり", "薄い", "受け入れ", "伺う", "うちわ", "腕前", "うるさい", "失う", "浮き袋", "裏切り", "麗らか", "売り場", "埋め立てる", "旨み", "薄手", "頷く", "動かす", "漆", "上向き", "生み出す", "売り切れ", "裏付け", "唸る", "浮かべる", "売り物", "疑う", "薄木", "打ち入り", "写し", "裏成り", "薄れる", "上澄み", "腕時計", "上の空", "牛蛙", "泡沫", "うねり", "裏表", "後ろ姿", "鬱陶しい", "上回る", "うつぶせ", "売り上げ", "空け", "海鳴り", "売り買い", "初々しい", "浮き沈み", "噂話", "上顎", "産毛", "鶯餅", "裏門", "売れ行き", "自惚れ", "上塗り", "上皮", "売れ残り", "薄氷", "上着", "薄暗い", "浮かれる", "鬱憤", "嬉しがる", "薄紙", "歌い手", "打ち合わせ", "丑三つ時", "薄絹", "空ろ", "うがい", "憂い", "唸く", "打ち掛ける", "うねる", "腕組み", "打ち寄せる", "上葉", "薄汚れ", "浮き足", "裏取り", "打ち出す", "裏口", "裏文字", "嘘泣き", "空木", "上物", "産着", "牛津", "鰻飯", "魚籠", "海の家", "打ち込み", "姥", "上野", "薄黒", "薄味",
"熊", "車", "薬", "口", "靴", "草", "雲", "首", "鯨", "果物", "国", "空気", "窪み", "海月", "茎", "胡桃", "黒", "空港", "籤", "楠", "くす玉", "籤引き", "唇", "繰り返し", "悔しい", "空間", "クレヨン", "鎖", "癖", "くしゃみ", "空白", "加える", "組む", "崩れる", "繰り上げる", "釘", "詳しい", "クリーム", "草むら", "草刈り", "暮らす", "空腹", "クリスマス", "クリスタル", "クリニック", "クローン", "クリーナー", "クッキー", "熊本", "クイーン", "クラシック", "踵", "栗きんとん", "クッキリ", "首輪", "砕ける", "暗闇", "くノ一", "腐る", "クタクタ", "桑原", "砕く", "苦労", "クエスト", "クリームパン", "くびれ", "熊啄木鳥", "クイズ", "空想", "口癖", "挫ける", "擽る", "クルー", "区切り", "潜る", "擽ったい", "草花", "九重", "暗い", "下る", "悔やむ", "窪む", "口籠もる", "燻製", "繰り上がる", "クラブ", "口説く", "くどい", "嘴", "苦しむ", "桑",
"素足", "水泳", "水産", "水準", "水素", "水中", "水筒", "水分", "水平", "睡眠", "水面", "水曜", "吸う", "数学", "数字", "数値", "崇拝", "末", "末っ子", "素顔", "姿", "好き", "隙間", "過ぎる", "すく", "救う", "少ない", "すぐ", "優れる", "スケート", "透ける", "凄い", "少し", "筋", "鈴", "濯ぐ", "進む", "裾", "スタジオ", "スタンド", "ステーキ", "素敵", "捨てる", "砂", "素直", "素晴らしい", "スピード", "全て", "滑る", "スポーツ", "スマート", "住まい", "墨", "菫", "住む", "相撲", "すやすや", "すらすら", "スリ", "鋭い", "すれ違う", "座る", "寸法", "スーツ", "スーパー", "スープ", "スムーズ", "スイーツ", "スイッチ", "水田", "水道", "炊飯", "水没", "推量", "スウィング", "スウェーデン",
"机", "翼", "梅雨", "積木", "土筆", "佃煮", "つくね", "作り物", "作り話", "作り事", "作り方", "作り替え", "作り出す", "作り上げる", "作り出し", "作る", "使う", "疲れる", "続く", "伝える", "包む", "繋ぐ", "呟く", "潰す", "摘む", "積もる", "強い", "辛い", "ツルツル", "連れる", "ツンツン", "蕾", "瞑る", "尽きる", "付き合う", "突き抜ける", "突き当たる", "突き止める", "突き進む",
"勇気", "郵便", "有名", "浴衣", "雪解け", "湯気", "漱ぐ", "譲る", "委ねる", "ゆっくり", "茹でる", "余裕", "輸入", "指差す", "夢見る", "湯浴み", "唯一", "遺言", "由緒", "憂鬱", "遊園地", "夕方", "勇敢", "夕暮れ", "幽谷", "優秀", "友人", "郵送", "誘導", "夕日", "昨夜", "夕焼け", "幽霊", "故に", "愉快", "縁", "歪む", "行き交う", "雪国", "雪だるま"],

                 "え段": ["駅", "絵本", "鉛筆", "枝", "映画", "笑顔", "英語", "選ぶ", "餌", "絵の具", "液体", "江ノ島", "襟巻き", "海老", "描く", "円盤", "衛星", "英雄", "襟足", "エアコン", "絵の具箱", "駅近", "遠足", "縁随", "枝豆", "縁側", "英文", "映像", "えこひいき", "園芸", "宴会", "婉曲", "英断", "会釈", "枝切り", "駅員", "永住", "煙突", "演劇", "栄養", "駅費", "えんどう", "偉そう", "縁付き", "江戸時代", "塩分", "閲覧", "沿岸", "英国", "縁起", "延長", "鉛筆削り", "絵巻", "衣文", "駅中", "衛生員", "偉ぶん", "縁日", "縁紙", "堰堤", "円高", "艶大な", "縁勝ち", "笑み", "営務", "遠近", "塩素", "永着", "餌入れ", "絵解き", "演説", "演じる", "演出", "永久", "円熟", "円滑", "円心", "援助", "炎症", "英文学", "縁談", "英知", "液化", "沿革", "円楽", "縁切り", "榎", "衛生", "映写", "遠距離", "遠方", "演曲", "演技", "演舞", "宴然", "円形", "演習", "艶色", "営業者", "演出家", "園芸家", "演技師",
"煙", "警察", "結婚", "怪我", "消しゴム", "系統", "稽古", "ケリ", "桁", "計算", "獣", "喧嘩", "建築", "毛虫", "蹴落とす", "見物", "鍵盤", "景色", "蹴散らす", "結果", "血圧", "血液", "研修", "研究", "今朝", "経験", "契約", "経費", "掲示板", "健康", "欠車", "賢者", "化粧", "結論", "検閲", "決定", "権", "県庁", "剣道", "建設", "計画", "計算機", "欠点", "警告", "健全", "権利", "系譜", "経営", "憲法", "蛍光灯", "権限", "痙攣", "警棒", "化粧品", "拳銃", "警戒", "軽蔑", "決心", "計算式", "蛍光ペン", "権威", "警官", "掲示", "検範囲", "倹約", "傾斜", "警文", "化粧室", "決議", "警察署", "健康診断", "欠席", "剣術", "欠如", "見解", "経験者", "結成", "快楽", "血縁", "経歴", "刑務所", "結晶", "警視動車", "警備", "境内", "研究室", "蹴込む", "形成", "警視", "健診", "軽傷", "系列", "検出", "敬意",
"性", "性格", "生活", "世紀", "請求", "清潔", "制限", "成功", "政策", "生産", "政治", "正式", "精神", "成人", "製造", "生存", "生態", "成長", "製鉄", "生徒", "青年", "整備", "政府", "生物", "成分", "製法", "生命", "制約", "西洋", "整理", "成立", "西暦", "正論", "咳", "石炭", "責任", "世間", "背筋", "世帯", "節", "設備", "切望", "説明", "節約", "背中", "狭い", "せめて", "攻める", "台詞", "世話", "線", "繊維", "先鋭", "戦果", "選挙", "宣言", "専攻", "繊細", "戦士", "選手", "扇子", "戦争", "選択", "船長", "選定", "宣伝", "仙人", "先輩", "扇風機", "選別", "洗面", "専門", "川柳", "先例", "セーター", "セール", "セーフティー", "セーラー", "背油", "制圧", "星雲", "声援", "西欧", "成果", "誓願", "正教", "整形", "制裁", "静止", "性質", "青春", "聖書", "清勝", "正常", "整数",
"手紙", "手袋", "天ぷら", "天国", "天井", "天皇", "天風", "転覆", "天気", "手伝う", "手洗い", "手当て", "手編み", "丁寧", "手掛ける", "適当", "手強い", "手探り", "手品", "手摺", "てっきり", "徹夜", "手抜き", "手放す", "手控え", "手前", "手回し", "照らす", "照れる", "手分け", "展開", "点検", "転校", "天才", "天使", "点数", "天体", "てんでん", "転倒", "天然",
"ヘビ", "へそ", "へんてこ", "へそくり", "へそまがり", "へそくりがま", "へそくりがめ", "へそくりばこ",
"迷惑", "目上", "恵む", "巡る", "目指す", "目下", "珍しい", "目立つ", "めちゃくちゃ", "メッキ", "めでたい", "目の前", "芽生え", "目まい", "メモリ", "明暗", "明快", "名曲", "名作", "名刺", "命じる", "瞑想", "名物", "名誉", "命令", "迷路", "雌牛", "目移り", "夫婦", "目方", "眼鏡", "恵まれる", "恵み", "巡り合う", "巡り合わせ", "目配せ", "目配り", "盲", "目黒", "目敏い", "目覚まし", "召し上がる", "召し抱える", "召使い", "雌しべ", "目印", "めだか", "目玉", "めちゃめちゃ", "滅多"],

                 "お段": ["音", "お菓子", "音楽", "お風呂", "お金", "お寺", "おやつ", "お店", "お箸", "お日様",
"お餅", "お花", "お父さん", "お母さん", "おじいさん", "おばあさん", "おもちゃ", "お弁当", "おかず", "折り紙",
"落とし物", "おしゃれ", "おにぎり", "お酒", "重い", "大きい", "美味しい", "面白い", "遅い", "大人",
"怒る", "踊る", "教える", "覚える", "終わる", "置く", "押す", "起こす", "降りる", "行う",
"思う", "恐れる", "怠る", "収まる", "抑える", "落ちる", "驚く", "送る", "思わず",
"おもてなし", "お祭り", "おせち", "おみくじ", "お土産", "お辞儀", "お任せ", "思い出", "お巡り",
"お宅", "お便り", "お手伝い", "お年玉", "踊り場", "お並び", "お話", "お参り", "趣", "親子",
"お休み", "お礼", "お詫び", "温泉", "温度", "応援", "王様", "王子", "横断", "往復",
"お陰", "小川", "掟", "屋上", "贈り物", "遅れ", "行い", "幼い", "惜しむ", "お互い",
"穏やか", "落ち葉", "夫", "男", "乙女", "脅し", "大人しい", "驚き", "お腹", "同じ",
"鬼ごっこ", "各々", "お早う", "帯", "お盆", "おまけ", "お守り", "お結び", "おめでとう", "面影",
"重さ", "表", "主に", "重り", "親子丼", "親父", "親指", "泳ぐ", "及ぼす", "織物",
"オルガン", "オレンジ", "卸し", "終わり", "恩返し", "恩恵", "温室", "温暖", "恩寵", "音符",
"お相", "追いかける", "追い越す", "生い茂る", "お出で", "追い抜く", "俺ら", "応じる", "応接", "応答",
"黄檗", "応募", "応用", "終える", "多い", "覆う", "大げさ", "大声", "大勢", "大空",
"概ね", "大物", "お帰り", "犯す", "女将", "拝む", "起き上がる", "置き換える", "置き去り", "置き時計",
"補う", "起きる", "奥様", "憶測", "奥地", "噯気", "送り仮名", "お稽古",
"厳か", "お小遣い", "収める", "惜しい", "おじいちゃん", "押し付け", "押し込む", "おじさん", "押し出す", "押し付ける",
"鴛鴦", "お釈迦", "お邪魔", "お正月", "押し寄せる", "白粉", "お勧め", "お世辞", "お世話", "襲う",
"恐らく", "恐れ", "恐ろしい",

"子供", "氷", "公園", "声", "米", "焦げ", "耕地", "苔", "心", "拳",
"交通", "焦げ茶", "炬燵", "小道", "国語", "こけし", "航空", "小鳥", "粉雪", "困った",
"交差点", "交番", "試み", "行動", "交換", "工業", "航空機", "小雨", "駒",
"腰掛け", "行進", "胡椒", "拵える", "コスモス", "答える", "紅茶", "乞食", "こっち", "小遣い",
"交際", "甑", "婚約", "今晩", "効果", "皇居", "高原", "言葉", "小麦粉",
"構成", "効力", "鉱物", "肯定", "高速", "柿葺", "子犬", "子狐", "古方",
"講義", "公式", "国際", "航空便", "鯉のぼり", "工場", "公害", "混合", "蝙蝠",
"購買", "校舎", "高級", "個別", "後輩", "交付", "行津", "小包",
"こんにちは", "こんばんは", "こちら", "今後", "混と", "国民", "梱包", "堪える",
"根気", "懲りる", "黒板", "小槌", "国士", "高校", "告白", "困る", "言者",

"そう", "相違", "憂鬱", "騒音", "創価", "総会", "総額", "送金", "遭遇", "送検",
"倉庫", "総合", "操作", "創作", "掃除", "葬式", "送信", "総数", "想像", "相続",
"早退", "相談", "装置", "想定", "相当", "遭難", "壮年", "相場", "総理", "添える",
"足", "即座", "促進", "即席", "測定", "速度", "速報", "側面", "測量", "底",
"祖国", "素材", "組織", "蘇生", "祖先", "注ぐ", "育てる", "措置", "そっくり", "そっと",
"袖", "外", "備える", "そのうち", "そのまま", "蕎麦", "祖父", "祖母", "粗末", "背く",
"染める", "そよ風", "それ", "それぞれ", "揃う", "揃える", "損", "損害", "尊敬", "存在",
"損失", "尊重", "ソース", "ソーセージ", "ソーダ", "ソープ", "粗悪", "そいつ", "草案", "創院",
"総円", "相応", "創刊", "早期", "総計", "創傑", "草原", "走行",

"時計", "鳥", "豆腐", "灯台", "トウモロコシ", "扉", "灯籠", "峠", "唐黍", "唐辛子",
"冬至", "東風", "冬季", "投票", "当選", "灯心", "遂に", "透明", "尊い", "闘将",
"遠い", "通す", "尖る", "解く", "得意", "特別", "溶ける", "閉じる", "戸棚", "途中",
"突然", "届く", "届ける", "留まる", "飛ぶ", "止まる", "止める", "友達", "捉える", "取り敢えず",

"ほうき", "ほたる", "ほうれんそう", "ほっぺた", "宝石", "ほうずき", "包丁人", "ほうれんそうごはん", "ほうれんそうサラダ", "ほうれんそうのおひたし",

"儲ける", "申す", "燃える", "潜る", "もしも", "もたれる", "勿論", "もっと", "もてなす", "戻す",
"物差し", "もはや", "もめる", "燃やす", "模様", "詣でる", "毛布", "萌黄", "もがく", "もぎ取る",
"目撃", "木材", "目次", "目的", "目標", "もぐさ", "もぐもぐ", "模型", "模索", "文字",
"若しくは", "もたらす", "餅", "持ち上げる", "持ち切り", "持ち越し", "持ち出す", "持ち直す", "持ち主", "持ち場",
"持つ", "もつれる", "もてあそぶ", "モテる", "求める", "もども", "物語", "物心", "物事", "物好き",

"よづり", "予定", "与党", "淀み", "夜中", "夜長", "夜泣き", "夜鍋", "余熱", "余念",
"よのぎ", "余輩", "余白", "夜明け", "よいしょ", "宵", "宵止め", "夜風", "予感", "預金",
"翌朝", "翌月", "翌日", "抑制", "欲望", "余計", "横切る", "寄越す", "汚れ", "予算",
"善し悪し", "予想", "予測", "余所見", "余所者", "よだれ", "よちよち", "四つ角", "四つ葉", "予定",
"夜通し", "夜な夜な", "夜逃げ", "余熱", "余白", "呼びかける", "呼び出す", "呼び止める", "余分", "予報",

"蝋燭", "録音", "論理", "朗読", "廊下", "路上", "論文", "老人", "労働", "六角",
"録画", "露店", "牢獄", "労費", "論争", "朗詠", "廊下", "老眼", "老朽", "労作",
"老女", "老衰", "蝋燭", "朗読", "浪人", "老婆", "老耄", "録画", "ろくでなし", "露骨"]


}

# 音韻パターン関連 - サンプルデータのみ

SOUND_REPETITION_TWISTERS = [
    "あかさたなはまやらわ",  # 五十音の「あ」行
    "いきしちにひみり",      # 五十音の「い」行
    "うくすつぬふむゆる",    # 五十音の「う」行
    "えけせてねへめれ",      # 五十音の「え」行
    "おこそとのほもよろを",  # 五十音の「お」行
    "かかかかかかかかか",    # 「か」の繰り返し
    "さささささささささ",    # 「さ」の繰り返し
    "たたたたたたたたた",    # 「た」の繰り返し
    "ななななななななな",    # 「な」の繰り返し
    "ははははははははは",    # 「は」の繰り返し
    "ままままままままま",    # 「ま」の繰り返し
    "ややややややややや",    # 「や」の繰り返し
    "ららららららららら",    # 「ら」の繰り返し
    "わわわわわわわわわ",    # 「わ」の繰り返し
    "ぱぴぷぺぽぱぴぷぺぽ",   # パ行の繰り返し
    "きゃきゅきょきゃきゅきょ", # キャ行の繰り返し
    "あいうえおあいうえお",   # 五十音の「あ」段繰り返し
    "らりるれろらりるれろ",   # ラ行の繰り返し
    "まみむめもまみむめも",   # マ行の繰り返し
    "ばびぶべぼばびぶべぼ"    # バ行の繰り返し
]
MORA_EXCHANGE_TWISTERS = [
    "とけいのけいと",    # 時計の計と
    "かみのみにか",      # 紙の実に香
    "さくらのくらさ",    # 桜の蔵さ
    "はなびのびはな",    # 花火の日花
    "みずのずみ",        # 水の図み
    "あめのめあ",        # 雨の目あ
    "ひかりのかひり",    # 光の香り
    "うみのみにう",      # 海の実に鵜
    "たけのけた",        # 竹の毛田
    "ほしのしほ",        # 星の詩帆
    "つきのきつ",        # 月の木津
    "なつのつな",        # 夏の綱
    "まくらのくらま",    # 枕の倉馬
    "そらのらそ",        # 空の良そ
    "いけのけい",        # 池の計
    "かおのおか",        # 顔の丘
    "とりのりと",        # 鳥の理と
    "くものもく",        # 雲の杢
    "やまのまや",        # 山の摩耶
    "ふねのねふ"         # 船の根布
]

CONSONANT_CLUSTER_TWISTERS = [
    "しゃしゅしょ",      # シャ行
    "ちゃちゅちょ",      # チャ行
    "きゃきゅきょ",      # キャ行
    "にゃにゅにょ",      # ニャ行
    "ひゃひゅひょ",      # ヒャ行
    "みゃみゅみょ",      # ミャ行
    "りゃりゅりょ",      # リャ行
    "ぎゃぎゅぎょ",      # ギャ行
    "じゃじゅじょ",      # ジャ行
    "びゃびゅびょ",      # ビャ行
    "ぴゃぴゅぴょ",      # ピャ行
    "きゃきゅきょにゃにゅにょ", # キャ行とニャ行の組み合わせ
    "しゃしゅしょちゃちゅちょ", # シャ行とチャ行の組み合わせ
    "ひゃひゅひょびゃびゅびょ", # ヒャ行とビャ行の組み合わせ
    "じゃじゅじょぎゃぎゅぎょ"  # ジャ行とギャ行の組み合わせ
]

SIMILAR_SOUND_TWISTERS = [
    "じゃじゅじょとざずぞ",      # ジャ行とザ行
    "たたたとだだだ",            # タとダ
    "かかかとががが",            # カとガ
    "さささとざざざ",            # サとザ
    "はははとばばば",            # ハとバ
    "ぱぱぱとばばば",            # パとバ
    "ちゃちゃちゃとじゃじゃじゃ", # チャとジャ
    "きききとぎぎぎ",            # キとギ
    "しししとじじじ",            # シとジ
    "すすすとずずず",            # スとズ
    "ふふふとぶぶぶ",            # フとブ
    "ひひひとびびび",            # ヒとビ
    "ぴぴぴとびびび",            # ピとビ
    "ちちちとぢぢぢ",            # チとヂ
    "つつつとづづづ"             # ツとヅ
]
PALINDROME_TWISTERS = [
    "たけやぶやけた",    # 竹薮焼けた
    "しんぶんし",        # 新聞紙
    "よるよるよ",        # 夜る夜るよ
    "まるまるま",        # 丸まるま
    "かみかみか",        # 紙噛みか
    "さかさかさ",        # 逆さかさ
    "たまたまた",        # たまたまた
    "はなはなは",        # 花はなは
    "らくらくら",        # 楽らくら
    "わかわかわ",        # 若わかわ
    "となりのりなと",    # 隣の利奈と
    "にわにわにわにわとり", # 庭に輪に和に鳥
    "わたしまけましたわ", # 私負けましたわ
    "やまかげやけがまや", # 山影焼け顔まや
    "くるまはまるく"     # 車は丸く
]

COUNTING_PATTERN_TWISTERS = ["一二三四五六七八九十"]

# 単語データ - サンプルデータのみ
NOUNS = [
    # 既存の名詞
    "猫", "犬", "鳥", "山", "川", "学校", "病院",

    # 自然物
    "空", "星", "太陽", "月", "雲", "雨", "雪", "風", "霧", "虹", "海", "砂", "森", "花", "草", "木", "葉",

    # 動物追加
    "熊", "狐", "兎", "蛇", "亀", "蝶", "蜂", "蟻", "カエル", "馬", "牛", "羊", "豚", "鹿", "猿", "キリン", "ライオン", "虎",

    # 場所追加
    "店", "映画館", "駅", "銀行", "郵便局", "美術館", "博物館", "動物園", "水族館", "道", "橋", "海岸", "砂浜", "森林", "畑", "庭", "遊園地", "城", "神社", "寺",

    # 日用品
    "本", "椅子", "机", "ペン", "鉛筆", "消しゴム", "紙", "時計", "電話", "カメラ", "テレビ", "ラジオ", "パソコン", "冷蔵庫", "洗濯機", "電子レンジ", "扇風機", "傘", "鍵", "靴", "帽子", "眼鏡", "鞄", "財布", "箸", "皿", "茶碗", "コップ", "鍋",

    # 食べ物追加
    "りんご", "みかん", "バナナ", "桃", "梨", "苺", "葡萄", "西瓜", "メロン", "チョコレート", "ケーキ", "クッキー", "アイスクリーム", "餅", "おにぎり", "焼きそば", "お好み焼き", "たこ焼き", "うどん", "そば", "天ぷら", "刺身", "納豆", "豆腐", "味噌汁",

    # 体の部位
    "頭", "顔", "目", "鼻", "口", "耳", "髪", "眉", "歯", "舌", "喉", "首", "肩", "腕", "肘", "手", "指", "爪", "胸", "背中", "腰", "お腹", "足", "膝",

    # 人・役割
    "人", "子供", "赤ちゃん", "少年", "少女", "大人", "男", "女", "お父さん", "お母さん", "おじいさん", "おばあさん", "兄", "姉", "弟", "妹", "友達", "先生", "医者", "看護師", "警察官", "消防士", "運転手", "料理人",

    # 抽象概念
    "愛", "平和", "自由", "幸せ", "喜び", "悲しみ", "怒り", "恐れ", "時間", "瞬間", "過去", "現在", "未来", "夢", "希望", "願い", "考え", "思い出", "言葉", "声", "音", "色", "形"
]

VERBS = [
    # 既存の動詞
    "食べる", "飲む", "走る", "泳ぐ", "見る",

    # 移動動詞
    "歩く", "登る", "降りる", "跳ぶ", "飛ぶ", "這う", "滑る", "転がる", "回る", "曲がる", "振る", "揺れる", "浮く", "沈む", "落ちる", "上がる", "下がる", "進む", "戻る", "通る", "過ぎる", "横切る", "追いかける", "逃げる", "隠れる",

    # 行動動詞
    "取る", "置く", "掴む", "投げる", "押す", "引く", "打つ", "叩く", "蹴る", "切る", "折る", "破る", "曲げる", "伸ばす", "結ぶ", "開ける", "閉める", "触る", "握る", "持つ", "運ぶ", "動かす", "止める", "探す", "見つける", "拾う", "与える", "受け取る", "捨てる", "集める",

    # 精神・感情動詞
    "考える", "思う", "覚える", "忘れる", "知る", "理解する", "信じる", "疑う", "期待する", "諦める", "望む", "決める", "選ぶ", "比べる", "変わる", "喜ぶ", "笑う", "泣く", "怒る", "恐れる", "驚く", "悩む", "困る", "後悔する", "安心する",

    # コミュニケーション動詞
    "話す", "聞く", "伝える", "教える", "答える", "質問する", "説明する", "命令する", "頼む", "謝る", "感謝する", "叫ぶ", "囁く", "呼ぶ", "歌う", "読む", "書く", "描く", "演じる", "踊る",

    # 日常動詞
    "起きる", "寝る", "着る", "脱ぐ", "座る", "立つ", "休む", "働く", "勉強する", "遊ぶ", "料理する", "洗濯する", "掃除する", "買い物する", "出かける", "帰る", "訪れる", "会う", "別れる", "待つ", "送る", "迎える", "作る", "修理する", "育てる", "成長する"
]

ADJECTIVES = [
    # 既存の形容詞
    "赤い", "青い", "大きい", "小さい", "暑い",

    # 色に関する形容詞
    "黄色い", "緑の", "紫の", "白い", "黒い", "茶色い", "ピンクの", "オレンジ色の", "金色の", "銀色の", "灰色の", "透明な", "虹色の", "蛍光色の", "明るい", "暗い",

    # 大きさ・形に関する形容詞
    "広い", "狭い", "長い", "短い", "高い", "低い", "深い", "浅い", "太い", "細い", "丸い", "四角い", "三角の", "平たい", "尖った", "曲がった", "真っ直ぐな", "歪んだ",

    # 質感に関する形容詞
    "硬い", "柔らかい", "滑らかな", "粗い", "ザラザラした", "ふわふわした", "べたべたした", "さらさらした", "ねばねばした", "つるつるした", "かさかさした", "じめじめした", "乾いた", "湿った",

    # 温度・天候に関する形容詞
    "寒い", "温かい", "冷たい", "熱い", "涼しい", "蒸し暑い", "ひんやりした", "ぬるい", "肌寒い", "酷暑の", "厳寒の", "雨降りの", "晴れた", "曇った", "風の強い", "霧の深い",

    # 味に関する形容詞
    "甘い", "辛い", "苦い", "酸っぱい", "しょっぱい", "美味しい", "まずい", "渋い", "香ばしい", "こってりした", "あっさりした", "濃厚な", "薄味の", "スパイシーな",

    # 感情・評価に関する形容詞
    "嬉しい", "悲しい", "怒った", "恐ろしい", "楽しい", "つまらない", "寂しい", "恥ずかしい", "誇らしい", "不安な", "心配な", "満足な", "落ち着いた", "焦った", "驚いた",

    # 状態に関する形容詞
    "新しい", "古い", "若い", "老いた", "綺麗な", "汚い", "賑やかな", "静かな", "忙しい", "暇な", "空いた", "混んだ", "整った", "散らかった", "壊れた", "完全な",

    # その他の形容詞
    "良い", "悪い", "素晴らしい", "ひどい", "優れた", "優しい", "厳しい", "強い", "弱い", "重い", "軽い", "速い", "遅い", "賢い", "愚かな", "面白い", "退屈な",
    "貴重な", "安価な", "便利な", "不便な", "危険な", "安全な", "健康的な", "不健康な", "正確な", "不正確な", "自然な", "人工的な", "特別な", "平凡な", "奇妙な"
]

FOOD_ITEMS = [
    # 既存の食べ物
    "寿司", "ラーメン", "カレー", "パン",

    # 和食（主食・麺類）
    "おにぎり", "茶碗蒸し", "お好み焼き", "たこ焼き", "うどん", "そば", "そうめん", "ひやむぎ", "焼きそば", "お茶漬け", "炊き込みご飯", "丼物", "親子丼", "牛丼", "天丼", "かつ丼", "天ぷらそば", "月見うどん", "きつねうどん",

    # 和食（おかず・一品料理）
    "刺身", "天ぷら", "唐揚げ", "とんかつ", "すき焼き", "しゃぶしゃぶ", "おでん", "焼き鳥", "照り焼き", "冷奴", "納豆", "豆腐", "卵焼き", "筑前煮", "肉じゃが", "きんぴらごぼう", "ひじき", "煮物", "焼き魚", "西京焼き", "あんかけ", "おひたし", "白和え",

    # 伝統和食
    "懐石料理", "精進料理", "雑煮", "おせち", "赤飯", "いなり寿司", "巻き寿司", "ちらし寿司", "押し寿司", "手巻き寿司", "甘酒", "茶碗蒸し", "かまぼこ",

    # 洋食・西洋料理
    "ハンバーグ", "オムライス", "コロッケ", "グラタン", "シチュー", "ビーフシチュー", "ポトフ", "ハヤシライス", "ピザ", "スパゲッティ", "ペペロンチーノ", "カルボナーラ", "ミートソース", "ピラフ", "フライドチキン", "サンドイッチ", "ホットドッグ", "パスタ", "ステーキ", "ハンバーガー", "スープ", "サラダ", "フレンチトースト",

    # 中華料理・アジア料理
    "餃子", "春巻き", "シュウマイ", "チャーハン", "麻婆豆腐", "酢豚", "八宝菜", "回鍋肉", "担々麺", "青椒肉絲", "エビチリ", "エビマヨ", "蒸し鶏", "小籠包", "肉まん", "あんまん", "中華丼", "フォー", "ビビンバ", "キムチ", "チヂミ", "タコス", "ナシゴレン", "ガパオライス", "パッタイ",

    # 郷土料理・ご当地グルメ
    "ちゃんぽん", "皿うどん", "明石焼き", "広島風お好み焼き", "もんじゃ焼き", "讃岐うどん", "稲庭うどん", "ほうとう", "石狩鍋", "ラフテー", "ゴーヤチャンプルー", "けんちん汁", "ひっつみ", "じゃじゃ麺", "わんこそば", "三色丼", "鮭とば", "きりたんぽ", "いかめし", "だんご汁", "佐賀牛", "名古屋コーチン", "ソースカツ丼",

    # スイーツ・デザート
    "羊羹", "大福", "団子", "ようかん", "饅頭", "どら焼き", "カステラ", "あんぱん", "クリームパン", "メロンパン", "シュークリーム", "プリン", "パフェ", "ケーキ", "チョコレート", "アイスクリーム", "かき氷", "ゼリー", "わらび餅", "たい焼き", "今川焼き", "最中", "おはぎ", "ぜんざい", "クレープ", "マカロン", "マフィン",

    # 果物・野菜料理
    "りんご", "みかん", "バナナ", "イチゴ", "メロン", "すいか", "ぶどう", "もも", "なし", "柿", "栗", "ブルーベリー", "キウイ", "パイナップル", "マンゴー", "アボカド", "さつまいも", "かぼちゃ", "なす", "トマト", "きゅうり", "もずく", "めかぶ", "とろろ", "山菜", "ぬか漬け", "野菜炒め",

    # 調味料・食材
    "醤油", "味噌", "塩", "砂糖", "わさび", "からし", "酢", "ごま", "海苔", "山葵", "七味", "ゆず", "柚子胡椒", "ポン酢", "マヨネーズ", "ソース", "ケチャップ", "ドレッシング", "バター", "マーガリン", "チーズ", "生クリーム", "ごま油", "オリーブオイル",

    # 飲み物
    "お茶", "緑茶", "抹茶", "ほうじ茶", "麦茶", "コーヒー", "紅茶", "ジュース", "牛乳", "豆乳", "ミルクティー", "カフェオレ", "カフェラテ", "ココア", "カルピス", "ラムネ", "サイダー", "コーラ", "炭酸水", "スムージー", "酒", "ビール", "日本酒", "焼酎", "ワイン", "梅酒", "カクテル",

    # ファストフード・軽食
    "フライドポテト", "からあげクン", "ポテトチップス", "おにぎり", "サンドイッチ", "おでん", "肉まん", "たこ焼き", "たい焼き", "コロッケ", "メンチカツ", "串カツ", "チーズフォンデュ", "ナゲット", "チキンライス", "チャーハン", "ピロシキ", "エンペラーズ", "シャウエッセン"
]


ADVERBS = [
    # 既存の副詞
    "とても", "非常に", "急に", "静かに",

    # 程度を表す副詞
    "かなり", "少し", "ほんの少し", "全く", "ほとんど", "あまり", "大いに", "極めて", "特に", "かろうじて", "ある程度", "相当", "著しく", "驚くほど",

    # 頻度を表す副詞
    "いつも", "よく", "時々", "たまに", "めったに", "決して", "常に", "頻繁に", "日々", "毎日", "四六時中", "何度も", "しばしば", "再三", "絶えず",

    # 方法を表す副詞
    "ゆっくり", "すばやく", "丁寧に", "乱暴に", "優しく", "強く", "弱く", "熱心に", "慎重に", "大胆に", "真剣に", "軽快に", "熱心に", "簡単に", "精密に",

    # 時間を表す副詞
    "今", "すぐに", "もうすぐ", "すでに", "まだ", "ついさっき", "以前", "後で", "まもなく", "しばらく", "ずっと", "突然", "徐々に", "直ちに", "瞬時に",

    # 場所を表す副詞
    "ここで", "そこで", "あそこで", "遠くで", "近くで", "前で", "後ろで", "上で", "下で", "内側で", "外側で", "周囲に", "中央に", "端に", "奥に",

    # 接続・論理に関する副詞
    "また", "さらに", "しかし", "けれども", "なぜなら", "したがって", "それでも", "確かに", "要するに", "実は", "つまり", "むしろ", "例えば", "特に", "結局",

    # その他の副詞
    "必ず", "絶対に", "もちろん", "おそらく", "たぶん", "せっかく", "あえて", "わざと", "偶然", "確実に", "仮に", "一応", "あいにく", "幸いに", "当然",
    "勿論", "思わず", "ついに", "意外と", "案外", "どうやら", "恐らく", "ひたすら", "本来", "単に", "むしろ", "うっかり", "何となく", "敢えて", "順調に"
]

PLACES = [
    # 既存の場所
    "公園", "学校", "病院", "家", "図書館",

    # 屋内施設
    "美術館", "博物館", "映画館", "レストラン", "カフェ", "ホテル", "デパート", "スーパー", "コンビニ", "オフィス", "工場", "体育館", "教室", "研究室", "寮",
    "アパート", "マンション", "居酒屋", "本屋", "八百屋", "魚屋", "肉屋", "パン屋", "洋服店", "靴屋", "文房具店", "薬局", "花屋", "ケーキ屋", "和菓子屋",

    # 公共施設
    "市役所", "区役所", "警察署", "消防署", "郵便局", "銀行", "裁判所", "刑務所", "大使館", "領事館", "保健所", "税務署", "年金事務所", "児童館", "公民館",

    # 教育施設
    "幼稚園", "保育園", "小学校", "中学校", "高校", "大学", "専門学校", "塾", "予備校", "職業訓練校", "大学院", "音楽学校", "美術学校", "調理学校", "語学学校",

    # 医療施設
    "クリニック", "診療所", "歯科医院", "眼科", "耳鼻科", "産婦人科", "小児科", "整形外科", "リハビリ施設", "老人ホーム", "介護施設", "健康センター", "動物病院",

    # 交通関連
    "駅", "空港", "港", "バス停", "タクシー乗り場", "駐車場", "高速道路", "踏切", "橋", "トンネル", "モノレール", "ロープウェイ", "フェリー乗り場", "地下鉄",

    # 自然・屋外
    "海", "川", "湖", "山", "森", "島", "砂浜", "砂漠", "草原", "滝", "洞窟", "温泉", "牧場", "水田", "畑", "果樹園", "湿原", "岬", "渓谷", "高原",

    # レジャー施設
    "動物園", "水族館", "遊園地", "競技場", "プール", "テーマパーク", "キャンプ場", "スキー場", "ゴルフ場", "テニスコート", "野球場", "サッカー場", "ボウリング場",

    # 文化施設
    "劇場", "ホール", "コンサートホール", "ライブハウス", "寄席", "ギャラリー", "プラネタリウム", "科学館", "歴史博物館", "民族博物館", "記念館", "資料館",

    # 宗教施設
    "神社", "寺", "教会", "モスク", "聖堂", "墓地", "霊園", "仏閣", "修道院", "礼拝堂", "大聖堂", "パゴダ", "祠", "御堂", "霊場",

    # 都市・地域
    "都市", "町", "村", "住宅街", "商店街", "繁華街", "工業地帯", "農村", "漁村", "郊外", "団地", "ニュータウン", "古都", "港町", "温泉街", "城下町"
]

SEASONS = [
    # 既存の季節
    "春", "夏", "秋", "冬",

    # 季節の細分化
    "初春", "晩春", "初夏", "盛夏", "晩夏", "初秋", "晩秋", "初冬", "厳冬", "晩冬",

    # 日本特有の季節区分
    "梅雨", "梅雨明け", "梅雨入り", "新緑", "紅葉", "落葉", "桜", "花見", "残暑", "台風", "小春",

    # 二十四節気関連
    "立春", "啓蟄", "春分", "清明", "立夏", "芒種", "夏至", "小暑", "立秋", "処暑", "秋分", "霜降", "立冬", "大雪", "冬至", "小寒",

    # 季節を表す別表現
    "春風", "春雨", "夏空", "夏風", "秋晴れ", "秋風", "冬景色", "冬日", "雪解け", "花冷え", "五月晴れ", "五月雨", "木枯らし", "師走",

    # 季節の祭りや行事
    "節分", "ひな祭り", "七夕", "お盆", "花火", "雪祭り", "彼岸", "端午", "七五三"
]

TIME_EXPRESSIONS = [
    # 既存の時間表現
    "朝", "昼", "夕方", "夜",

    # 一日の詳細な区分
    "早朝", "明け方", "午前", "正午", "午後", "日暮れ", "日没", "宵", "真夜中", "未明", "深夜", "夜明け", "黎明", "黄昏", "夕暮れ", "夕刻",

    # 周期的な時間
    "毎朝", "毎昼", "毎晩", "日々", "毎日", "毎週", "毎月", "毎年", "平日", "週末", "休日",

    # 曜日
    "月曜", "火曜", "水曜", "木曜", "金曜", "土曜", "日曜",

    # 月
    "一月", "二月", "三月", "四月", "五月", "六月", "七月", "八月", "九月", "十月", "十一月", "十二月",

    # 相対的な時間表現
    "今", "昨日", "明日", "先週", "来週", "先月", "来月", "去年", "今年", "来年", "さっき", "すぐに", "もうすぐ", "間もなく", "いつか", "ついさっき",

    # 時間的距離
    "昔", "大昔", "古代", "将来", "未来", "過去", "現在", "今後", "永遠", "瞬間", "一瞬", "一時", "暫く", "長時間", "短時間", "刹那",

    # 時間に関する慣用表現
    "夜通し", "終日", "一昼夜", "徹夜", "明け暮れ", "朝夕", "朝な夕な", "夜な夜な", "四六時中", "昼夜", "今日明日", "旦夕"
]


ANIMALS = [
    # 既存の動物
    "猫", "犬", "鳥", "魚", "象",

    # 哺乳類（家畜・ペット）
    "馬", "牛", "豚", "羊", "山羊", "ウサギ", "ハムスター", "モルモット", "フェレット",

    # 哺乳類（野生動物）
    "狼", "狐", "熊", "鹿", "猿", "リス", "たぬき", "イタチ", "ムササビ", "モモンガ", "アライグマ", "カピバラ", "コアラ", "カンガルー", "パンダ",

    # 大型・特徴的な哺乳類
    "キリン", "カバ", "ライオン", "虎", "豹", "チーター", "ジャガー", "シマウマ", "サイ", "バッファロー", "イノシシ", "オオカミ", "ラクダ", "ナマケモノ", "アルマジロ", "アリクイ",

    # 海洋哺乳類
    "クジラ", "イルカ", "シャチ", "アザラシ", "セイウチ", "ラッコ", "ジュゴン", "マナティー",

    # 鳥類
    "スズメ", "カラス", "鷹", "鷲", "フクロウ", "ハト", "ニワトリ", "アヒル", "白鳥", "カモ", "キジ", "クジャク", "ペンギン", "フラミンゴ", "ツバメ", "カナリア", "オウム", "インコ", "ダチョウ", "ホトトギス", "うぐいす", "九官鳥",

    # 爬虫類・両生類
    "蛇", "トカゲ", "イグアナ", "カメレオン", "亀", "ワニ", "カエル", "ヒキガエル", "イモリ", "サンショウウオ",

    # 魚類
    "鮭", "鯖", "鮪", "鯛", "鰯", "鱈", "鮎", "金魚", "鯉", "鰻", "ふぐ", "鮫", "エイ", "サメ", "クジラ", "マンボウ", "タツノオトシゴ",

    # 昆虫・節足動物
    "蝶", "蛾", "蜂", "蟻", "蝉", "トンボ", "バッタ", "カマキリ", "コオロギ", "カブトムシ", "クワガタ", "テントウムシ", "ゴキブリ", "クモ", "サソリ", "ムカデ",

    # その他の水生動物
    "タコ", "イカ", "クラゲ", "ヒトデ", "ウニ", "貝", "エビ", "カニ", "ロブスター", "サンゴ",

    # 絶滅動物・伝説の生物
    "恐竜", "マンモス", "ドードー", "竜", "鳳凰", "麒麟", "ユニコーン", "グリフィン", "ケルベロス",

    # 日本特有の動物や伝説の生物
    "狸", "河童", "天狗", "鬼", "妖狐", "鶴", "雉", "猪", "鷺", "雀", "鮒", "鯨", "油虫"
]

PARTICLES = [
    # 既存の助詞
    "が", "を", "に", "は", "も", "と", "で",

    # 格助詞
    "へ", "から", "まで", "より", "にて", "について", "において",

    # 係助詞
    "こそ", "さえ", "すら", "だけ", "ばかり", "のみ", "しか", "など", "くらい", "ほど", "きり", "っきり",

    # 並列助詞
    "や", "やら", "だの", "とか", "なり", "にも",

    # 接続助詞
    "ながら", "つつ", "ので", "のに", "から", "ため", "けれど", "けれども", "が", "けど", "し", "ものの", "ところで",

    # 終助詞
    "か", "かい", "かしら", "かな", "かね", "がな", "な", "なあ", "ね", "ねえ", "よ", "わ", "ぞ", "ぜ", "さ", "っけ", "もの", "もん",

    # 間投助詞
    "ね", "な", "さ", "よ", "や",

    # 副助詞
    "くらい", "ぐらい", "ほど", "だけ", "しか", "ばかり", "まで", "など", "なんか", "なんて", "でも", "だって",

    # 複合助詞
    "によって", "について", "に関して", "に対して", "にとって", "として", "によると", "をもって", "をめぐって", "にわたって",

    # 特殊な助詞
    "って", "じゃ", "っては", "っちゃ", "ってば", "ったら"
]

CONJUNCTIONS = [
    # 既存の接続詞
    "と", "や", "また", "そして",

    # 並列接続詞
    "および", "かつ", "ならびに", "とともに", "のみならず", "しかも", "それに", "おまけに", "その上", "なおかつ",

    # 累加接続詞
    "それから", "そのうえ", "そのほか", "さらに", "加えて", "次に", "続いて", "ついで", "あとは", "最後に",

    # 転換接続詞
    "さて", "ところで", "では", "それでは", "ただ", "しかし", "だが", "けれども", "だけど", "でも", "ところが", "それでも", "そのくせ", "一方", "逆に",

    # 順接接続詞
    "だから", "それで", "すると", "したがって", "そのため", "ゆえに", "それゆえ", "よって", "それなら", "すなわち", "それでは",

    # 逆説接続詞
    "しかし", "ところが", "だが", "しかしながら", "それなのに", "なのに", "にもかかわらず", "それにしても",

    # 説明接続詞
    "つまり", "すなわち", "要するに", "換言すれば", "言い換えれば", "例えば", "たとえば", "特に", "とりわけ", "なかでも",

    # 補足接続詞
    "なお", "ちなみに", "ただし", "もっとも", "ただ", "一方", "他方", "むしろ", "そもそも", "実は",

    # 選択接続詞
    "あるいは", "または", "もしくは", "それとも", "なり",

    # 条件接続詞
    "もし", "万一", "仮に", "たとえ", "いっそ", "たといまたは",

    # 時間関係接続詞
    "そのとき", "すると", "そのうちに", "やがて", "そのあと", "それから", "すでに", "ついに",

    # 理由・原因接続詞
    "なぜなら", "というのは", "というのも", "なんとなれば", "だって"
]


# 意味のある文のパターン - サンプルデータのみ
MEANINGFUL_PATTERNS = [
    # 既存のパターン
    "{animal}が{food}を食べる",
    "{place}で{animal}が{verb}",

    # 基本的な主語・述語パターン
    "{animal}が{verb}",
    "{animal}は{adjective}",
    "{animal}と{animal}が{verb}",
    "{animal}は{animal}より{adjective}",
    "{noun}が{verb}ている",
    "{animal}が{animal}を{verb}",

    # 場所・時間を含むパターン
    "{place}で{animal}が{adverb}{verb}",
    "{place}の{noun}が{verb}",
    "{time_expression}に{place}で{verb}",
    "{season}の{place}で{animal}が{verb}",
    "{place}から{place}まで{verb}",
    "{place}で{food}を{verb}",

    # 形容詞を含むパターン
    "{adjective}{noun}が{adjective}{noun}を{verb}",
    "{animal}の{adjective}{noun}",
    "{adjective}{place}で{adjective}{food}を{verb}",
    "{adjective}{animal}と{adjective}{animal}",
    "{season}の{adjective}{noun}",

    # 複文や重文を含むパターン
    "{animal}が{verb}と{animal}が{verb}",
    "{place}で{verb}ながら{verb}",
    "{animal}は{verb}けれども{animal}は{verb}",
    "{time_expression}に{verb}たら{noun}が{verb}",
    "{adjective}{noun}が{verb}ので{noun}が{verb}",

    # 副詞を含むパターン
    "{adverb}{verb}ている{animal}",
    "{animal}が{adverb}{verb}",
    "{adverb}{adjective}{noun}が{adverb}{verb}",
    "{place}で{adverb}{adverb}{verb}",

    # 季節感を含むパターン
    "{season}の{place}で{animal}が{food}を{verb}",
    "{season}になると{animal}が{verb}",
    "{season}の{adjective}{noun}が{verb}",
    "{season}の{time_expression}に{place}で{verb}",

    # 会話的な表現を含むパターン
    "{animal}が「{adverb}{verb}」と{verb}",
    "「{adjective}{noun}だ」と{animal}が{verb}",
    "{animal}に「{place}で{verb}」と{verb}",

    # 疑問文パターン
    "{animal}は何を{verb}のか",
    "{place}で{animal}が{verb}のはなぜ",
    "どうして{adjective}{animal}が{verb}のか",
    "{animal}と{animal}はどちらが{adjective}のか",

    # 命令文パターン
    "{adverb}{place}で{verb}なさい",
    "{adjective}{food}を{verb}てください",
    "{animal}に{food}を{verb}させて",

    # 慣用句・ことわざ風パターン
    "{animal}の{noun}も{noun}のうち",
    "{place}の{animal}は{place}の{food}を{verb}",
    "{adjective}{animal}も{noun}から{verb}",
    "{season}の{animal}は{adjective}",

    # 三すくみパターン（三つの要素の関係）
    "{animal}は{animal}に勝ち、{animal}は{animal}に勝つ",
    "{place}の{animal}、{place}の{food}、{place}の{noun}",
    "{adjective}{noun}、{adjective}{noun}、{adjective}{noun}",

    # 対比パターン
    "{animal}は{verb}が、{animal}は{verb}ない",
    "{place}では{verb}、{place}では{verb}ない",
    "{adjective}{noun}と{adjective}{noun}の違い",

    # 言葉遊びパターン
    "{noun}の{noun}による{noun}のための{noun}",
    "{animal}と{animal}と{animal}が{verb}",
    "まるで{animal}のような{animal}が{verb}",
    "{noun}のような{noun}が{noun}のように{verb}"
]


# 単語の組み合わせルール - サンプルデータのみ
WORD_COMBINATION_RULES = {
    # 既存のルール
    "寿司": {
        "verbs": ["食べる", "握る", "並べる", "注文する", "振る舞う", "詰める", "巻く", "盛る", "提供する"],
        "nouns": ["ネタ", "シャリ", "わさび", "ガリ", "イクラ", "マグロ", "板前", "回転", "セット", "握り", "軍艦", "手巻き"],
        "adjectives": ["美味しい", "新鮮な", "握りたての", "華やかな", "高級な", "創作", "伝統的な", "豪華な", "職人技の"]
    },
    "本": {
        "verbs": ["読む", "書く", "めくる", "閉じる", "借りる", "装丁する", "積む", "集める", "片付ける", "整理する"],
        "nouns": ["表紙", "ページ", "章", "著者", "出版社", "内容", "背表紙", "帯", "挿絵", "活字", "文庫", "新刊"],
        "adjectives": ["面白い", "分厚い", "古い", "新しい", "貴重な", "読みやすい", "難解な", "有名な", "希少な", "重厚な"]
    },

    # 食べ物カテゴリ
    "ラーメン": {
        "verbs": ["啜る", "食べる", "茹でる", "作る", "持ち帰る", "冷やす", "温める", "混ぜる"],
        "nouns": ["麺", "スープ", "具", "チャーシュー", "メンマ", "ねぎ", "海苔", "店", "屋台", "丼"],
        "adjectives": ["熱い", "美味しい", "こってりした", "あっさりした", "濃厚な", "細麺の", "太麺の", "塩味の", "醤油味の"]
    },
    "お好み焼き": {
        "verbs": ["焼く", "返す", "切る", "混ぜる", "注ぐ", "食べる", "作る", "盛る"],
        "nouns": ["生地", "キャベツ", "具", "鉄板", "ヘラ", "ソース", "マヨネーズ", "かつお節", "青のり"],
        "adjectives": ["熱々の", "ふわふわの", "カリカリの", "広島風の", "関西風の", "モダン", "もちもちの"]
    },
    "カレー": {
        "verbs": ["煮込む", "食べる", "混ぜる", "かける", "作る", "盛る", "温める", "辛くする"],
        "nouns": ["ルー", "スパイス", "具", "お皿", "ナン", "福神漬け", "ターメリック", "チキン", "ポーク"],
        "adjectives": ["辛い", "甘い", "濃厚な", "本格的な", "インド風の", "日本風の", "スパイシーな", "まろやかな"]
    },

    # 飲み物カテゴリ
    "コーヒー": {
        "verbs": ["飲む", "淹れる", "豆を挽く", "注ぐ", "かき混ぜる", "冷やす", "温める", "提供する"],
        "nouns": ["豆", "カップ", "ドリッパー", "ミル", "フィルター", "アロマ", "苦味", "酸味", "泡", "エスプレッソ"],
        "adjectives": ["香ばしい", "苦い", "熱い", "アイスの", "ブラックの", "深煎りの", "浅煎りの", "濃い", "薄い"]
    },
    "お茶": {
        "verbs": ["飲む", "淹れる", "注ぐ", "楽しむ", "提供する", "出す", "冷やす", "温める"],
        "nouns": ["茶葉", "急須", "湯のみ", "茶道", "和菓子", "苦味", "渋み", "香り", "抹茶", "煎茶"],
        "adjectives": ["苦い", "渋い", "香ばしい", "甘い", "濃い", "薄い", "熱い", "冷たい", "優雅な", "伝統的な"]
    },

    # 動物カテゴリ
    "猫": {
        "verbs": ["鳴く", "寝る", "歩く", "じゃれる", "飛びつく", "隠れる", "撫でる", "遊ぶ"],
        "nouns": ["毛", "尻尾", "ひげ", "爪", "肉球", "首輪", "ミルク", "キャットフード", "猫じゃらし"],
        "adjectives": ["可愛い", "小さい", "大きい", "毛深い", "甘えん坊の", "気まぐれな", "好奇心旺盛な", "眠そうな"]
    },
    "犬": {
        "verbs": ["吠える", "走る", "咥える", "舐める", "追いかける", "散歩する", "躾ける", "遊ぶ"],
        "nouns": ["毛", "尻尾", "耳", "牙", "リード", "首輪", "骨", "犬小屋", "ドッグフード", "飼い主"],
        "adjectives": ["忠実な", "元気な", "賢い", "大きい", "小さい", "毛深い", "愛らしい", "活発な", "従順な"]
    },

    # 場所カテゴリ
    "公園": {
        "verbs": ["歩く", "遊ぶ", "走る", "座る", "休む", "散歩する", "集まる", "待ち合わせる"],
        "nouns": ["ベンチ", "遊具", "木", "花", "池", "芝生", "道", "噴水", "広場", "花壇"],
        "adjectives": ["広い", "静かな", "賑やかな", "緑豊かな", "美しい", "都会の", "住宅地の", "歴史的な"]
    },
    "図書館": {
        "verbs": ["読む", "借りる", "返す", "調べる", "探す", "静かにする", "勉強する", "通う"],
        "nouns": ["本", "書架", "カウンター", "司書", "閲覧室", "雑誌", "新聞", "参考書", "カード", "コンピューター"],
        "adjectives": ["静かな", "広い", "古い", "新しい", "便利な", "公共の", "学術的な", "充実した", "近代的な"]
    },

    # 自然カテゴリ
    "山": {
        "verbs": ["登る", "降りる", "眺める", "歩く", "越える", "写真を撮る", "探検する", "キャンプする"],
        "nouns": ["頂上", "斜面", "岩", "森", "道", "渓谷", "滝", "霧", "雪", "景色", "富士山"],
        "adjectives": ["高い", "険しい", "美しい", "雄大な", "神秘的な", "静かな", "深い", "広大な", "神聖な"]
    },
    "海": {
        "verbs": ["泳ぐ", "潜る", "眺める", "浮かぶ", "釣りをする", "波に乗る", "散歩する", "遊ぶ"],
        "nouns": ["波", "砂浜", "岩", "貝殻", "潮", "水平線", "日没", "魚", "サンゴ", "ヨット"],
        "adjectives": ["青い", "広い", "静かな", "荒れた", "深い", "美しい", "透明な", "広大な", "冷たい"]
    },

    # 季節・気象カテゴリ
    "雨": {
        "verbs": ["降る", "止む", "しのぐ", "打つ", "濡らす", "流れる", "聞く", "避ける"],
        "nouns": ["傘", "雨粒", "雷", "虹", "水たまり", "梅雨", "スコール", "雨雲", "雨音", "レインコート"],
        "adjectives": ["強い", "弱い", "冷たい", "しとしとした", "土砂降りの", "小降りの", "突然の", "季節の"]
    },
    "春": {
        "verbs": ["訪れる", "過ごす", "楽しむ", "感じる", "迎える", "待つ", "彩る", "告げる"],
        "nouns": ["桜", "花見", "新緑", "芽吹き", "卒業", "入学", "春風", "春雨", "ひな祭り", "春分"],
        "adjectives": ["温かい", "穏やかな", "柔らかい", "明るい", "希望に満ちた", "新鮮な", "活気ある", "美しい"]
    },

    # 文化・芸術カテゴリ
    "映画": {
        "verbs": ["観る", "撮る", "演じる", "制作する", "上映する", "鑑賞する", "評価する", "楽しむ"],
        "nouns": ["俳優", "監督", "脚本", "撮影", "音楽", "特殊効果", "スクリーン", "観客", "チケット", "ポップコーン"],
        "adjectives": ["面白い", "感動的な", "迫力ある", "ロマンチックな", "ホラーな", "コメディな", "アクション満載の", "長い"]
    },
    "音楽": {
        "verbs": ["聴く", "演奏する", "歌う", "作曲する", "録音する", "踊る", "楽しむ", "癒される"],
        "nouns": ["楽器", "メロディ", "リズム", "歌詞", "音符", "和音", "コンサート", "楽団", "歌手", "DJ"],
        "adjectives": ["美しい", "心地よい", "激しい", "穏やかな", "ロックな", "クラシックな", "ジャズっぽい", "伝統的な"]
    },

    # スポーツカテゴリ
    "野球": {
        "verbs": ["打つ", "投げる", "捕る", "走る", "守る", "応援する", "観戦する", "プレーする"],
        "nouns": ["バット", "ボール", "グローブ", "ピッチャー", "キャッチャー", "塁", "ホームラン", "スタジアム", "チーム", "監督"],
        "adjectives": ["白熱した", "スリリングな", "熱狂的な", "プロの", "アマチュアの", "激しい", "伝統的な", "エキサイティングな"]
    },
    "水泳": {
        "verbs": ["泳ぐ", "潜る", "浮く", "練習する", "競争する", "教える", "鍛える", "楽しむ"],
        "nouns": ["プール", "水着", "ゴーグル", "クロール", "平泳ぎ", "バタフライ", "背泳ぎ", "コーチ", "選手", "記録"],
        "adjectives": ["速い", "上手な", "熟練した", "水に強い", "オリンピックの", "競技用の", "健康的な", "爽快な"]
    },

    # 日用品カテゴリ
    "傘": {
        "verbs": ["差す", "畳む", "開く", "持つ", "忘れる", "貸す", "借りる", "しまう"],
        "nouns": ["骨", "布", "柄", "雨", "日差し", "レイングッズ", "折りたたみ", "ビニール傘", "日傘", "和傘"],
        "adjectives": ["赤い", "黒い", "大きな", "小さな", "丈夫な", "お洒落な", "便利な", "カラフルな", "高級な"]
    },
    "時計": {
        "verbs": ["見る", "合わせる", "巻く", "止まる", "動く", "進む", "遅れる", "身につける"],
        "nouns": ["針", "文字盤", "ベルト", "電池", "アラーム", "秒針", "分針", "時針", "機械", "腕時計"],
        "adjectives": ["正確な", "高級な", "古い", "新しい", "アナログの", "デジタルの", "壊れた", "遅れている", "シンプルな"]
    },

    # 電化製品カテゴリ
    "スマホ": {
        "verbs": ["使う", "見る", "操作する", "充電する", "触る", "持ち歩く", "通話する", "撮影する"],
        "nouns": ["画面", "カメラ", "アプリ", "電池", "通知", "ケース", "充電器", "データ", "タッチパネル", "通信"],
        "adjectives": ["新しい", "古い", "高機能の", "便利な", "壊れた", "軽い", "スマートな", "高価な", "スリムな"]
    },
    "テレビ": {
        "verbs": ["見る", "つける", "消す", "録画する", "選局する", "調整する", "置く", "組み立てる"],
        "nouns": ["画面", "チャンネル", "リモコン", "番組", "音声", "ドラマ", "ニュース", "スポーツ中継", "CM", "映像"],
        "adjectives": ["大きい", "小さい", "薄型の", "液晶の", "鮮明な", "有機ELの", "スマートな", "壊れた", "最新式の"]
    },

    # その他のカテゴリ
    "夢": {
        "verbs": ["見る", "叶える", "追いかける", "語る", "実現する", "描く", "忘れる", "思い描く"],
        "nouns": ["希望", "目標", "未来", "願い", "理想", "憧れ", "幻", "ビジョン", "イメージ", "睡眠"],
        "adjectives": ["大きな", "小さな", "美しい", "儚い", "明るい", "暗い", "不思議な", "現実的な", "非現実的な"]
    },
    "言葉": {
        "verbs": ["話す", "書く", "伝える", "選ぶ", "聞く", "覚える", "忘れる", "理解する"],
        "nouns": ["意味", "表現", "単語", "文法", "発音", "会話", "文章", "詩", "小説", "コミュニケーション"],
        "adjectives": ["美しい", "難しい", "簡単な", "深い", "複雑な", "直接的な", "婉曲的な", "古風な", "現代的な"]
    },
    "思い出": {
        "verbs": ["振り返る", "語る", "思い出す", "大切にする", "忘れる", "残す", "共有する", "記録する"],
        "nouns": ["写真", "日記", "出来事", "記憶", "瞬間", "旅行", "学生時代", "幼少期", "友人", "家族"],
        "adjectives": ["懐かしい", "美しい", "楽しい", "悲しい", "輝かしい", "忘れられない", "大切な", "若き日の"]
    },
    "旅行": {
        "verbs": ["行く", "楽しむ", "計画する", "準備する", "予約する", "探検する", "帰る", "写真を撮る"],
        "nouns": ["目的地", "スーツケース", "地図", "パスポート", "ホテル", "観光地", "土産", "飛行機", "電車", "バス"],
        "adjectives": ["楽しい", "遠い", "近い", "国内の", "海外の", "冒険的な", "リラックスした", "思い出深い"]
    }
}


# ※※※ ここまで単語データ部分（後で詳細なデータに置き換え） ※※※

#############################################################
#                クラス・関数定義部分                        #
#############################################################

class TongueTwisterGenerator:
    def __init__(self):
        """初期化メソッド：音声ファイルのクリーンアップと設定初期化を行う"""
        # 前回の音声ファイルをクリーンアップ
        self._cleanup_previous_audio()

        # 設定を全て読み込む
        self.initialize_settings()

    def _cleanup_previous_audio(self):
        """前回の実行で生成された音声ファイルを削除する"""
        try:
            # "twister_*.mp3" パターンに一致するファイルを検索して削除
            audio_files = glob.glob("twister_*.mp3")
            for file in audio_files:
                os.remove(file)
                print(f"前回の音声ファイルを削除しました: {file}")

            # 一時ディレクトリの音声ファイルも削除
            temp_dir = tempfile.gettempdir()
            temp_audio_files = glob.glob(os.path.join(temp_dir, "tongue_twister_*.mp3"))
            for file in temp_audio_files:
                os.remove(file)
        except Exception as e:
            print(f"音声ファイルのクリーンアップに失敗しました: {e}")

    def initialize_settings(self):
        """設定を初期化するメソッド：各種データをインスタンス変数に格納"""
        # ユーザー設定値を読み込む
        self.output_count = OUTPUT_COUNT
        self.difficulty_level = DIFFICULTY_LEVEL
        self.selected_dialects = SELECTED_DIALECTS
        self.enable_auto_generate = ENABLE_AUTO_GENERATE
        self.generation_mode = GENERATION_MODE
        self.pattern_weights = PATTERN_WEIGHTS
        self.speech_settings = SPEECH_SETTINGS

        # 基本の早口言葉
        self.classic_twisters = CLASSIC_TWISTERS

        # 難読熟語・言いにくい言葉
        self.difficult_phrases = DIFFICULT_PHRASES

        # 方言の早口言葉
        self.dialect_twisters = DIALECT_TWISTERS

        # 行別・段別の言葉グループ
        self.row_words = ROW_WORDS
        self.column_words = COLUMN_WORDS

        # 音韻パターン関連の単語グループ
        self.sound_repetition_twisters = SOUND_REPETITION_TWISTERS
        self.mora_exchange_twisters = MORA_EXCHANGE_TWISTERS
        self.consonant_cluster_twisters = CONSONANT_CLUSTER_TWISTERS

        # 特殊パターンの早口言葉
        self.similar_sound_twisters = SIMILAR_SOUND_TWISTERS
        self.palindrome_twisters = PALINDROME_TWISTERS
        self.counting_pattern_twisters = COUNTING_PATTERN_TWISTERS

        # バリエーション拡張用の単語データ
        self.nouns = NOUNS
        self.verbs = VERBS
        self.adjectives = ADJECTIVES
        self.adverbs = ADVERBS
        self.places = PLACES
        self.seasons = SEASONS
        self.time_expressions = TIME_EXPRESSIONS
        self.food_items = FOOD_ITEMS
        self.animals = ANIMALS
        self.particles = PARTICLES
        self.conjunctions = CONJUNCTIONS

        # 意味のある文のパターン
        self.meaningful_patterns = MEANINGFUL_PATTERNS

        # 単語の組み合わせルール
        self.word_combination_rules = WORD_COMBINATION_RULES

        # 難易度レベルマッピング
        self.difficulty_levels = DIFFICULTY_LEVELS

        # 拡張早口言葉リストを初期化
        self.expanded_twisters = []

        # 早口言葉を生成
        self.generate_expanded_twisters()

        # 組み合わせ数を計算して表示
        self.calculate_combinations()

    def generate_expanded_twisters(self):
        """多様な早口言葉パターンを生成する"""
        # 拡張早口言葉リストを初期化
        self.expanded_twisters = []

        # 選択されたモードに基づいて早口言葉を生成
        if self.generation_mode == "standard":
            # 標準モード：すべての種類の早口言葉を含む（統合モード）
            self._add_standard_twisters()
        elif self.generation_mode == "dialect_only":
            # 方言のみモード：選択された方言の早口言葉だけを使用
            self._add_dialect_twisters()
        elif self.generation_mode == "classic_only":
            # 古典のみモード：定番の早口言葉だけを使用
            self.expanded_twisters.extend(self.classic_twisters)
        elif self.generation_mode == "sound_patterns":
            # 音韻パターンモード：音の特徴を活かした早口言葉を使用
            self._add_sound_pattern_twisters()
        elif self.generation_mode == "row_column":
            # 行・段パターンモード：同じ行や段の音を組み合わせた早口言葉を使用
            self._add_row_column_twisters()
        elif self.generation_mode == "meaningful":
            # 意味のある早口言葉モード：文法的に正しい意味のある早口言葉だけを生成
            self._add_meaningful_twisters()
        elif self.generation_mode == "word_rules":
            # 単語組み合わせルールモード：定義されたルールに基づいて早口言葉を生成
            self._add_word_rule_twisters()

        # 自動生成が有効で、かつ標準モードやmeaningfulモードの場合、追加のパターンを生成
        if self.enable_auto_generate and self.generation_mode in ["standard", "meaningful", "word_rules"]:
            self._generate_additional_patterns()

        # 生成された早口言葉をランダムに並べ替え（種類によるグループ化を避ける）
        random.shuffle(self.expanded_twisters)

        # 不自然な早口言葉をフィルタリング
        self._filter_unnatural_twisters()

        # 拡張完了後の確認
        print(f"早口言葉バリエーション数: {len(self.expanded_twisters)}個")

    def _filter_unnatural_twisters(self):
        """不自然な早口言葉をフィルタリングする"""
        filtered_twisters = []

        # 無意味な回文パターンのリスト
        nonsense_palindromes = [
            "ナポリタンたんりぽな", "けーききーけ", "ネコじゃじこね", "バナナなば",
            "カレーでれーか", "十二支とじゅうにし"
        ]

        # 不自然なひらがなのみの文のパターンの一部
        unnatural_hiragana_patterns = [
            "たこのまどでなつを持つ", "えびをおとはうみ", "いすをきつねはみず",
            "いすのきつねでひとを買う", "きつねがかぜがくま", "へびのえきとてら",
            "ひとのちずでみずをある", "きつねのかぜでけんを買う",
            "ゆきのつきでふねを買う", "ふねのゆきですしをいる",
            "けむりのねこでてらをある"
        ]

        for twister in self.expanded_twisters:
            # 無意味な回文をスキップ
            if twister in nonsense_palindromes:
                continue

            # 特定の不自然なひらがなのみの文をスキップ
            if twister in unnatural_hiragana_patterns:
                continue

            # クラクララの繰り返しのようなパターンをスキップ
            if "クララがこれからさがるからこれからさがるからこれがクララ" in twister:
                continue

            # 単語の無理矢理な回文（逆さ言葉）をスキップ
            bad_palindromes = ["たんりぽな", "きーけ", "じゃじこね", "なば", "れーか"]
            skip_this = False
            for bad in bad_palindromes:
                if bad in twister:
                    skip_this = True
                    break
            if skip_this:
                continue

            filtered_twisters.append(twister)

        self.expanded_twisters = filtered_twisters

    def calculate_combinations(self):
        """早口言葉の理論上の組み合わせ数を計算する"""
        # 基本組み合わせパターン数
        basic_combinations = len(self.classic_twisters) + len(self.difficult_phrases)

        # 方言パターン数
        dialect_combinations = sum(len(phrases) for phrases in self.dialect_twisters.values())

        # 音韻パターン数
        sound_pattern_combinations = (
            len(self.sound_repetition_twisters) +
            len(self.mora_exchange_twisters) +
            len(self.consonant_cluster_twisters) +
            len(self.similar_sound_twisters) +
            len(self.palindrome_twisters) +
            len(self.counting_pattern_twisters)
        )

        # 行・段パターン
        row_column_combinations = 0
        for row in self.row_words:
            words = self.row_words[row]
            if len(words) >= 2:
                row_column_combinations += (len(words) * (len(words) - 1)) // 2

        for column in self.column_words:
            words = self.column_words[column]
            if len(words) >= 2:
                row_column_combinations += (len(words) * (len(words) - 1)) // 2

        # 単語組み合わせルール
        word_rule_combinations = len(self.word_combination_rules) * 10

        # 自動生成可能な組み合わせ数
        auto_generated_combinations = 0
        if self.enable_auto_generate:
            # 名詞+助詞+動詞パターン
            auto_generated_combinations += len(self.nouns) * len(self.particles) * len(self.verbs)

            # 形容詞+名詞パターン（単体および繰り返し）
            auto_generated_combinations += len(self.adjectives) * len(self.nouns) * 2

            # 名詞の列挙パターン（2個および3個）
            auto_generated_combinations += (
                len(self.nouns) * (len(self.nouns) - 1) +
                len(self.nouns) * (len(self.nouns) - 1) * (len(self.nouns) - 2) // 6
            )

            # 場所+時間+動詞パターン
            auto_generated_combinations += len(self.places) * len(self.time_expressions) * len(self.verbs)

            # 副詞+動詞パターン
            auto_generated_combinations += len(self.adverbs) * len(self.verbs)

            # 季節+形容詞+名詞パターン
            auto_generated_combinations += len(self.seasons) * len(self.adjectives) * len(self.nouns)

            # 動物+助詞+食べ物+動詞パターン
            auto_generated_combinations += len(self.animals) * len(self.particles) * len(self.food_items) * len(self.verbs)

            # 意味のあるパターン数
            auto_generated_combinations += len(self.meaningful_patterns) * 10  # 各パターンに対して約10の変数

        # 合計組み合わせ数
        total_combinations = basic_combinations + dialect_combinations + sound_pattern_combinations + row_column_combinations + word_rule_combinations
        if self.enable_auto_generate:
            total_combinations += auto_generated_combinations

        # 結果を表示
        print("\n=== 早口言葉の組み合わせ数 ===")
        print(f"基本パターン: {basic_combinations}通り")
        print(f"方言パターン: {dialect_combinations}通り")
        print(f"音韻パターン: {sound_pattern_combinations}通り")
        print(f"行・段パターン: {row_column_combinations}通り")
        print(f"単語ルール: {word_rule_combinations}通り")
        if self.enable_auto_generate:
            print(f"自動生成可能: {auto_generated_combinations}通り")
        print(f"理論上の合計: {total_combinations}通り")
        print("===========================")

        return total_combinations

    def _add_standard_twisters(self):
        """標準的な早口言葉を追加（統合版）"""
        # 既存の早口言葉を追加
        self.expanded_twisters.extend(self.classic_twisters)
        self.expanded_twisters.extend(self.difficult_phrases)

        # 選択された方言のみ追加
        for dialect in self.selected_dialects:
            if dialect in self.dialect_twisters:
                self.expanded_twisters.extend(self.dialect_twisters[dialect])

        # 難易度に応じたリストの追加
        difficulty_index = self.difficulty_levels.index(self.difficulty_level)

        # 簡単な早口言葉は全難易度で含める
        self.expanded_twisters.extend(self.sound_repetition_twisters)

        # 普通以上の難易度では複雑な早口言葉を追加
        if difficulty_index >= 1:  # 普通以上
            self.expanded_twisters.extend(self.mora_exchange_twisters)
            self.expanded_twisters.extend(self.palindrome_twisters)
            self.expanded_twisters.extend(self.counting_pattern_twisters)

        # 難しい以上の難易度ではさらに難しい早口言葉を追加
        if difficulty_index >= 2:  # 難しい以上
            self.expanded_twisters.extend(self.consonant_cluster_twisters)
            self.expanded_twisters.extend(self.similar_sound_twisters)

        # 超難しい以上は最大の難易度
        if difficulty_index >= 3:  # 超難しい以上
            # 追加の難しい組み合わせを生成
            for i in range(30):
                twister1 = random.choice(self.difficult_phrases)
                twister2 = random.choice(self.consonant_cluster_twisters)
                self.expanded_twisters.append(f"{twister1}、{twister2}")

        # 【統合機能１】行・段パターンの早口言葉を追加（数を増加）
        self._generate_row_column_patterns(200)  # 200個の行・段パターンを追加

        # 【統合機能２】意味のある文のパターンを追加
        self._generate_meaningful_patterns(150)  # 150個の意味のある文を追加

        # 【統合機能３】単語組み合わせルールを使った早口言葉を追加
        self._generate_word_rule_patterns(150)  # 150個の単語ルールパターンを追加

    def _add_dialect_twisters(self):
        """方言のみの早口言葉を追加"""
        # 選択された方言の早口言葉を追加
        all_dialect_twisters = []
        for dialect in self.selected_dialects:
            if dialect in self.dialect_twisters:
                all_dialect_twisters.extend(self.dialect_twisters[dialect])

        # 選択した方言がない場合は、すべての方言を使用
        if not all_dialect_twisters:
            for dialect_phrases in self.dialect_twisters.values():
                all_dialect_twisters.extend(dialect_phrases)

        self.expanded_twisters.extend(all_dialect_twisters)

    def _add_sound_pattern_twisters(self):
        """音韻パターンの早口言葉を追加"""
        self.expanded_twisters.extend(self.sound_repetition_twisters)
        self.expanded_twisters.extend(self.mora_exchange_twisters)
        self.expanded_twisters.extend(self.consonant_cluster_twisters)
        self.expanded_twisters.extend(self.palindrome_twisters)

    def _add_row_column_twisters(self):
        """行・段パターンの早口言葉を追加"""
        # 目標生成数
        target_count = 200

        # 行パターンの早口言葉を生成
        self._generate_row_patterns(target_count // 2)

        # 段パターンの早口言葉を生成
        self._generate_column_patterns(target_count // 2)

    def _add_meaningful_twisters(self):
        """意味のある早口言葉を追加"""
        # 基本の早口言葉（意味がある）を追加
        self.expanded_twisters.extend(self.classic_twisters)

        # 方言の早口言葉（意味がある）を追加
        for dialect in self.selected_dialects:
            if dialect in self.dialect_twisters:
                self.expanded_twisters.extend(self.dialect_twisters[dialect])

        # 意味のある文のパターンで生成
        self._generate_meaningful_patterns(200)

    def _add_word_rule_twisters(self):
        """単語組み合わせルールを使った早口言葉を追加"""
        # 基本の早口言葉を追加
        self.expanded_twisters.extend(self.classic_twisters)

        # 単語ルールを使った早口言葉を生成
        self._generate_word_rule_patterns(200)

    def _generate_row_patterns(self, count):
        """同じ行の音を使った早口言葉を生成"""
        for _ in range(count):
            # ランダムに行を選択
            row_key = random.choice(list(self.row_words.keys()))
            row_words = self.row_words[row_key]

            if len(row_words) >= 3:
                # その行から3つの単語をランダムに選択
                selected_words = random.sample(row_words, 3)

                # パターンタイプを選択
                pattern_type = random.randint(1, 3)

                if pattern_type == 1:
                    # パターン1: 単語を並べる
                    new_twister = f"{selected_words[0]}、{selected_words[1]}、{selected_words[2]}"
                elif pattern_type == 2:
                    # パターン2: 助詞で結合
                    particles = ["が", "を", "に", "は", "と", "の"]
                    particle1 = random.choice(particles)
                    particle2 = random.choice(particles)
                    new_twister = f"{selected_words[0]}{particle1}{selected_words[1]}{particle2}{selected_words[2]}"
                else:
                    # パターン3: 意味のある文章形式
                    verb = random.choice(["ある", "いる", "見る", "買う", "持つ"])
                    new_twister = f"{selected_words[0]}の{selected_words[1]}で{selected_words[2]}を{verb}"

                # 単語の組み合わせが不自然でないかチェック
                if self._check_meaning_simple(selected_words):
                    if new_twister not in self.expanded_twisters:
                        self.expanded_twisters.append(new_twister)

    def _generate_column_patterns(self, count):
        """同じ段の音を使った早口言葉を生成"""
        for _ in range(count):
            # ランダムに段を選択
            column_key = random.choice(list(self.column_words.keys()))
            column_words = self.column_words[column_key]

            if len(column_words) >= 3:
                # その段から3つの単語をランダムに選択
                selected_words = random.sample(column_words, 3)

                # パターンタイプを選択
                pattern_type = random.randint(1, 3)

                if pattern_type == 1:
                    # パターン1: 単語を並べる
                    new_twister = f"{selected_words[0]}、{selected_words[1]}、{selected_words[2]}"
                elif pattern_type == 2:
                    # パターン2: 助詞で結合
                    particles = ["が", "を", "に", "は", "と", "の"]
                    particle1 = random.choice(particles)
                    particle2 = random.choice(particles)
                    new_twister = f"{selected_words[0]}{particle1}{selected_words[1]}{particle2}{selected_words[2]}"
                else:
                    # パターン3: 意味のある文章形式
                    verb = random.choice(["ある", "いる", "見る", "買う", "持つ"])
                    new_twister = f"{selected_words[0]}の{selected_words[1]}で{selected_words[2]}を{verb}"

                # 単語の組み合わせが不自然でないかチェック
                if self._check_meaning_simple(selected_words):
                    if new_twister not in self.expanded_twisters:
                        self.expanded_twisters.append(new_twister)

    def _generate_row_column_patterns(self, count):
        """行と段を組み合わせたパターンを生成"""
        # 行パターンと段パターンを均等に生成
        row_count = count // 2
        column_count = count - row_count

        self._generate_row_patterns(row_count)
        self._generate_column_patterns(column_count)

    def _generate_meaningful_patterns(self, count):
        """意味のある文のパターンを生成"""
        for _ in range(count):
            # ランダムにパターンを選択
            pattern = random.choice(self.meaningful_patterns)

            # パターンに合わせて単語を選択
            if "{animal}" in pattern:
                pattern = pattern.replace("{animal}", random.choice(self.animals))
            if "{food}" in pattern:
                pattern = pattern.replace("{food}", random.choice(self.food_items))
            if "{place}" in pattern:
                pattern = pattern.replace("{place}", random.choice(self.places))
            if "{time}" in pattern:
                pattern = pattern.replace("{time}", random.choice(self.time_expressions))
            if "{season}" in pattern:
                pattern = pattern.replace("{season}", random.choice(self.seasons))
            if "{adj}" in pattern:
                pattern = pattern.replace("{adj}", random.choice(self.adjectives))
            if "{noun}" in pattern:
                pattern = pattern.replace("{noun}", random.choice(self.nouns))
            if "{verb}" in pattern:
                pattern = pattern.replace("{verb}", random.choice(self.verbs))
            if "{adverb}" in pattern:
                pattern = pattern.replace("{adverb}", random.choice(self.adverbs))

            # 生成された文を追加
            if pattern not in self.expanded_twisters:
                self.expanded_twisters.append(pattern)

    def _generate_word_rule_patterns(self, count):
        """単語組み合わせルールを使った早口言葉を生成"""
        for _ in range(count):
            # ランダムに単語を選択
            keyword = random.choice(list(self.word_combination_rules.keys()))

            # 選択した単語に対するルールを取得
            rule = self.word_combination_rules[keyword]

            # ルールに基づいた単語を選択
            if 'verbs' in rule:
                verb = random.choice(rule['verbs'])
            else:
                verb = random.choice(self.verbs)

            if 'nouns' in rule:
                noun = random.choice(rule['nouns'])
            else:
                noun = random.choice(self.nouns)

            if 'adjectives' in rule:
                adj = random.choice(rule['adjectives'])
            else:
                adj = random.choice(self.adjectives)

            # パターンを選択して文を生成
            pattern_type = random.randint(1, 3)

            if pattern_type == 1:
                new_twister = f"{keyword}を{verb}"
            elif pattern_type == 2:
                new_twister = f"{adj}{keyword}を{verb}"
            else:
                particle = random.choice(self.particles)
                new_twister = f"{noun}{particle}{keyword}を{verb}"

            # 生成された文を追加
            if new_twister not in self.expanded_twisters:
                self.expanded_twisters.append(new_twister)

    def _generate_adj_noun_patterns(self, count):
        """形容詞+名詞パターン（「形容詞+名詞」を最大2回繰り返し）を生成"""
        for _ in range(count):
            adj = random.choice(self.adjectives)
            noun = random.choice(self.nouns)

            # 形容詞と名詞の組み合わせが自然かチェック
            if not self._is_natural_adj_noun(adj, noun):
                continue

            repeat_count = random.randint(1, 2)  # 繰り返し回数を1〜2回に制限

            if repeat_count == 1:
                new_twister = f"{adj}{noun}"
            else:
                new_twister = f"{adj}{noun}、{adj}{noun}"

            # 単語の組み合わせが不自然でないかチェック
            if self._check_meaning_simple([adj, noun]):
                if new_twister not in self.expanded_twisters:
                    self.expanded_twisters.append(new_twister)

    def _generate_noun_list_patterns(self, count):
        """名詞リストパターン（2〜3個の名詞を組み合わせる）を生成"""
        for _ in range(count):
            noun_count = random.randint(2, 3)
            nouns = random.sample(self.nouns, noun_count)

            # 名詞のカテゴリを確認（同じカテゴリの名詞を選ぶ）
            if not self._are_related_nouns(nouns):
                continue

            if noun_count == 2:
                new_twister = f"{nouns[0]}と{nouns[1]}"
            else:
                new_twister = f"{nouns[0]}と{nouns[1]}と{nouns[2]}"

            # 単語の組み合わせが不自然でないかチェック
            if self._check_meaning_simple(nouns):
                if new_twister not in self.expanded_twisters:
                    self.expanded_twisters.append(new_twister)

    def _generate_noun_verb_patterns(self, count):
        """名詞+助詞+動詞パターンを生成"""
        for _ in range(count):
            noun = random.choice(self.nouns)
            verb = random.choice(self.verbs)
            particle = random.choice(self.particles)

            # 助詞と動詞の自然な組み合わせをチェック
            if self._is_natural_combination(noun, particle, verb):
                new_twister = f"{noun}{particle}{verb}"

                # 単語の組み合わせが不自然でないかチェック
                if self._check_meaning_simple([noun, verb]):
                    if new_twister not in self.expanded_twisters:
                        self.expanded_twisters.append(new_twister)

    def _generate_place_time_patterns(self, count):
        """場所+時間+動詞パターンを生成"""
        for _ in range(count):
            place = random.choice(self.places)
            time = random.choice(self.time_expressions)
            verb = random.choice(self.verbs)

            # 場所と動詞の組み合わせが自然かチェック
            if not self._is_natural_place_verb(place, verb):
                continue

            # ランダムなパターンを選択
            pattern_type = random.randint(1, 3)

            if pattern_type == 1:
                new_twister = f"{place}で{time}{verb}"
            elif pattern_type == 2:
                new_twister = f"{time}に{place}で{verb}"
            else:
                new_twister = f"{place}で{verb}{time}"

            # 単語の組み合わせが不自然でないかチェック
            if self._check_meaning_simple([place, time, verb]):
                if new_twister not in self.expanded_twisters:
                    self.expanded_twisters.append(new_twister)

    def _generate_adverb_verb_patterns(self, count):
        """副詞+動詞パターンを生成"""
        for _ in range(count):
            adverb = random.choice(self.adverbs)
            verb = random.choice(self.verbs)

            # 副詞と動詞の組み合わせが自然かチェック
            if not self._is_natural_adverb_verb(adverb, verb):
                continue

            new_twister = f"{adverb}{verb}"

            # 単語の組み合わせが不自然でないかチェック
            if self._check_meaning_simple([adverb, verb]):
                if new_twister not in self.expanded_twisters:
                    self.expanded_twisters.append(new_twister)

    def _generate_season_patterns(self, count):
        """季節+形容詞+名詞パターンを生成"""
        for _ in range(count):
            season = random.choice(self.seasons)
            adj = random.choice(self.adjectives)
            noun = random.choice(self.nouns)

            # 季節と形容詞の組み合わせが自然かチェック
            if not self._is_natural_season_adj(season, adj):
                continue

            pattern_type = random.randint(1, 2)

            if pattern_type == 1:
                new_twister = f"{season}の{adj}{noun}"
            else:
                new_twister = f"{season}は{adj}{noun}の季節"

            # 単語の組み合わせが不自然でないかチェック
            if self._check_meaning_simple([season, adj, noun]):
                if new_twister not in self.expanded_twisters:
                    self.expanded_twisters.append(new_twister)

    def _generate_animal_food_patterns(self, count):
        """動物+助詞+食べ物+動詞パターンを生成"""
        for _ in range(count):
            animal = random.choice(self.animals)
            food = random.choice(self.food_items)
            verb = random.choice(["食べる", "見る", "欲しがる", "嫌う"])

            # 動物と食べ物の組み合わせが自然かチェック
            if not self._is_natural_animal_food(animal, food, verb):
                continue

            # 自然な組み合わせに限定
            if verb == "食べる" or verb == "欲しがる":
                new_twister = f"{animal}が{food}を{verb}"
            elif verb == "見る":
                new_twister = f"{animal}が{food}を{verb}"
            else:  # 嫌う
                new_twister = f"{animal}は{food}を{verb}"

            # 単語の組み合わせが不自然でないかチェック
            if self._check_meaning_simple([animal, food, verb]):
                if new_twister not in self.expanded_twisters:
                    self.expanded_twisters.append(new_twister)

    def _generate_combined_patterns(self, count):
        """既存パターンの組み合わせを生成"""
        for _ in range(count):
            twister1 = random.choice(self.classic_twisters)
            twister2 = random.choice([t for t in self.classic_twisters if t != twister1])

            # 接続詞をランダムに選択
            conjunction = random.choice(self.conjunctions)

            new_twister = f"{twister1}{conjunction}{twister2}"

            # 古典的な早口言葉の組み合わせは意味チェック不要

            if new_twister not in self.expanded_twisters:
                self.expanded_twisters.append(new_twister)

    def _generate_additional_patterns(self):
        """追加の早口言葉パターンを生成"""
        target_count = 1000  # 最終的な目標数

        # 追加パターン生成が必要な場合のみ実行
        if len(self.expanded_twisters) < target_count:
            patterns_needed = target_count - len(self.expanded_twisters)

            # 各パターンタイプの配分を計算
            pattern_types = list(self.pattern_weights.keys())
            random.shuffle(pattern_types)  # ランダムな順序で生成

            for pattern_type in pattern_types:
                count = int(patterns_needed * self.pattern_weights[pattern_type])

                if pattern_type == "adj_noun_pattern":
                    self._generate_adj_noun_patterns(count)
                elif pattern_type == "noun_list_pattern":
                    self._generate_noun_list_patterns(count)
                elif pattern_type == "noun_particle_verb_pattern":
                    self._generate_noun_verb_patterns(count)
                elif pattern_type == "place_time_pattern":
                    self._generate_place_time_patterns(count)
                elif pattern_type == "adverb_verb_pattern":
                    self._generate_adverb_verb_patterns(count)
                elif pattern_type == "season_adj_noun_pattern":
                    self._generate_season_patterns(count)
                elif pattern_type == "animal_food_pattern":
                    self._generate_animal_food_patterns(count)
                elif pattern_type == "combined_pattern":
                    self._generate_combined_patterns(count)
                elif pattern_type == "row_column_pattern":
                    self._generate_row_column_patterns(count)
                elif pattern_type == "meaningful_pattern":
                    self._generate_meaningful_patterns(count)
                elif pattern_type == "word_rule_pattern":
                    self._generate_word_rule_patterns(count)

    def _is_natural_combination(self, noun, particle, verb):
        """名詞+助詞+動詞の組み合わせが自然かどうかをチェック"""
        # 一部の不自然な組み合わせを除外
        if particle == "を" and verb in ["ある", "いる", "なる"]:
            return False
        if particle == "が" and verb in ["食べる", "飲む", "読む", "見る"]:
            return False
        if particle == "で" and verb in ["ある", "いる", "なる", "傷つく"]:
            return False

        # 特定の不自然な組み合わせをチェック
        unnatural_combinations = [
            ("きつね", "が", "いぬ"),
            ("りんご", "に", "季節"),
            ("旅館", "で", "悲しむ"),
            ("カメ", "が", "走る"),
            ("象", "を", "持つ"),
            ("山", "が", "飲む"),
            ("テレビ", "が", "泳ぐ"),
            ("机", "が", "雪を食べる"),
            ("地図", "で", "伸ばす")
        ]

        for bad_noun, bad_particle, bad_verb in unnatural_combinations:
            if noun == bad_noun and particle == bad_particle and verb == bad_verb:
                return False

        return True

    def _is_natural_adj_noun(self, adj, noun):
        """形容詞+名詞の組み合わせが自然かどうかをチェック"""
        # 特定の不自然な組み合わせをチェック
        unnatural_combinations = [
            ("赤い", "海"),
            ("青い", "ご飯"),
            ("嬉しい", "山"),
            ("悲しい", "机"),
            ("寒い", "夏"),
            ("熱い", "冬"),
            ("透明な", "歌"),
            ("暗い", "太陽"),
            ("眠い", "机")
        ]

        for bad_adj, bad_noun in unnatural_combinations:
            if adj == bad_adj and noun == bad_noun:
                return False

        return True

    def _are_related_nouns(self, nouns):
        """名詞が関連しているかどうかをチェック"""
        # カテゴリ別の名詞グループ
        noun_categories = {
            "動物": ["猫", "犬", "鳥", "魚", "虫", "馬", "牛", "羊", "豚", "鶏", "象", "鹿", "狐", "狸", "熊", "猿", "兎", "蛙", "蛇", "亀"],
            "自然": ["山", "川", "海", "空", "雲", "雨", "雪", "風", "星", "月", "太陽", "地球", "宇宙", "森", "林", "砂漠", "湖", "島"],
            "場所": ["学校", "病院", "会社", "工場", "店", "駅", "空港", "公園", "動物園", "水族館", "図書館", "美術館", "博物館", "映画館", "銀行"],
            "道具": ["本", "ペン", "紙", "鉛筆", "消しゴム", "定規", "ノート", "地図", "辞書", "新聞", "雑誌", "手紙", "携帯", "パソコン", "カメラ"]
        }

        # 3つ以上の名詞がある場合、最初の2つだけをチェック
        check_nouns = nouns[:2] if len(nouns) > 2 else nouns

        # 同じカテゴリの名詞かどうかをチェック
        for category, category_nouns in noun_categories.items():
            if all(noun in category_nouns for noun in check_nouns):
                return True

        # 異なるカテゴリでも、一部の組み合わせは自然
        natural_pairs = [
            ("猫", "魚"),
            ("犬", "骨"),
            ("鳥", "虫"),
            ("学校", "本"),
            ("病院", "医者"),
            ("店", "商品"),
            ("寿司","シャリ")
        ]

        if len(check_nouns) == 2 and tuple(check_nouns) in natural_pairs:
            return True

        # デフォルトでは75%の確率で関連しているとみなす（バリエーションを増やすため）
        return random.random() < 0.75

    def _is_natural_place_verb(self, place, verb):
        """場所+動詞の組み合わせが自然かどうかをチェック"""
        # 特定の不自然な組み合わせをチェック
        unnatural_combinations = [
            ("病院", "笑う"),
            ("学校", "寝る"),
            ("銀行", "泳ぐ"),
            ("駅", "料理する"),
            ("図書館", "叫ぶ"),
            ("映画館", "料理する"),
            ("病院", "踊る")
        ]

        for bad_place, bad_verb in unnatural_combinations:
            if place == bad_place and verb == bad_verb:
                return False

        # 場所ごとの自然な動詞リスト
        natural_place_verbs = {
            "学校": ["学ぶ", "教える", "読む", "書く", "考える"],
            "病院": ["治療する", "診察する", "待つ", "休む"],
            "公園": ["遊ぶ", "歩く", "走る", "座る", "休む"],
            "図書館": ["読む", "調べる", "勉強する", "探す"],
            "レストラン": ["食べる", "飲む", "注文する", "支払う"],
            "映画館": ["見る", "楽しむ", "感動する"]
        }

        # 場所に対する自然な動詞リストがあれば、その中に動詞があるかチェック
        if place in natural_place_verbs and verb in natural_place_verbs[place]:
            return True

        # デフォルトでは80%の確率で自然とみなす
        return random.random() < 0.8

    def _is_natural_adverb_verb(self, adverb, verb):
        """副詞+動詞の組み合わせが自然かどうかをチェック"""
        # 特定の不自然な組み合わせをチェック
        unnatural_combinations = [
            ("静かに", "叫ぶ"),
            ("大声で", "囁く"),
            ("ゆっくり", "急ぐ"),
            ("速く", "休む"),
            ("丁寧に", "壊す"),
            ("猛烈に", "寝る"),
            ("静々と", "走る")
        ]

        for bad_adverb, bad_verb in unnatural_combinations:
            if adverb == bad_adverb and verb == bad_verb:
                return False

        # 副詞ごとの自然な動詞リスト
        natural_adverb_verbs = {
            "静かに": ["話す", "読む", "考える", "歩く", "寝る"],
            "大声で": ["叫ぶ", "歌う", "話す", "笑う"],
            "急に": ["止まる", "変わる", "起きる", "現れる"],
            "優しく": ["触れる", "話す", "撫でる", "扱う"]
        }

        # 副詞に対する自然な動詞リストがあれば、その中に動詞があるかチェック
        if adverb in natural_adverb_verbs and verb in natural_adverb_verbs[adverb]:
            return True

        # デフォルトでは80%の確率で自然とみなす
        return random.random() < 0.8

    def _is_natural_season_adj(self, season, adj):
        """季節+形容詞の組み合わせが自然かどうかをチェック"""
        # 特定の不自然な組み合わせをチェック
        unnatural_combinations = [
            ("夏", "寒い"),
            ("冬", "暑い"),
            ("春", "枯れた"),
            ("秋", "新緑の"),
            ("夏", "白い"),
            ("冬", "熱帯の"),
            ("春", "雪深い")
        ]

        for bad_season, bad_adj in unnatural_combinations:
            if season == bad_season and adj == bad_adj:
                return False

        # 季節ごとの自然な形容詞リスト
        natural_season_adjs = {
            "春": ["暖かい", "若々しい", "明るい", "優しい", "さわやかな", "新しい"],
            "夏": ["暑い", "熱い", "まぶしい", "活発な", "元気な", "鮮やかな"],
            "秋": ["涼しい", "静かな", "落ち着いた", "黄色い", "赤い", "豊かな"],
            "冬": ["寒い", "冷たい", "白い", "静かな", "厳しい", "澄んだ"]
        }

        # 季節に対する自然な形容詞リストがあれば、その中に形容詞があるかチェック
        if season in natural_season_adjs and adj in natural_season_adjs[season]:
            return True

        # デフォルトでは75%の確率で自然とみなす
        return random.random() < 0.75

    def _is_natural_animal_food(self, animal, food, verb):
        """動物+食べ物+動詞の組み合わせが自然かどうかをチェック"""
        # 特定の不自然な組み合わせをチェック
        unnatural_combinations = [
            ("猫", "バナナ", "食べる"),
            ("犬", "寿司", "食べる"),
            ("鳥", "ラーメン", "食べる"),
            ("馬", "チョコレート", "食べる"),
            ("魚", "お肉", "食べる"),
            ("猫", "コーヒー", "飲む"),
            ("象", "ケーキ", "作る")
        ]

        for bad_animal, bad_food, bad_verb in unnatural_combinations:
            if animal == bad_animal and food == bad_food and verb == bad_verb:
                return False

        # 動物ごとの自然な食べ物リスト
        natural_animal_foods = {
            "猫": ["魚", "猫缶", "ミルク", "キャットフード"],
            "犬": ["肉", "骨", "ドッグフード"],
            "鳥": ["虫", "種", "果物", "パン"],
            "魚": ["小魚", "エサ", "藻"]
        }

        # 動物に対する自然な食べ物リストがあれば、その中に食べ物があるかチェック
        if animal in natural_animal_foods and food in natural_animal_foods[animal]:
            return True

        # デフォルトでは70%の確率で自然とみなす
        return random.random() < 0.7

    def _check_meaning_simple(self, words):
        """簡易的な意味チェック - 単語の組み合わせが不自然でないかをチェック"""
        # 意味のない組み合わせリスト
        meaningless_combinations = [
            ["きつね", "いぬ", "りんご"],
            ["季節", "旅館", "悲しむ"],
            ["カメ", "走る", "電話"],
            ["象", "持つ", "パソコン"],
            ["山", "飲む", "本"],
            ["地図", "伸ばす", "透明"],
            ["テレビ", "泳ぐ", "読む"],
            ["机", "雪", "炒める"],
            ["銀行", "歌う", "空"],
            ["魚", "歩く", "ペン"]
        ]

        # words内の単語が意味のない組み合わせに含まれていないかチェック
        for combination in meaningless_combinations:
            matched = 0
            for word in words:
                if word in combination:
                    matched += 1
                if matched >= 2:  # 2つ以上の単語が一致した場合は不自然
                    return False

        return True

    def generate(self, count=None):
        """指定された数の早口言葉をランダムに生成する"""
        if count is None:
            count = self.output_count

        # expanded_twistersからランダムに選択
        if len(self.expanded_twisters) < count:
            return self.expanded_twisters
        return random.sample(self.expanded_twisters, count)

    def generate_audio(self, text, filename=None, speed=1.0, play_in_colab=True):
        """早口言葉の音声ファイルを生成し、Colab上で再生する"""
        if filename is None:
            # 一時ファイル名を生成
            temp_dir = tempfile.gettempdir()
            filename = os.path.join(temp_dir, f"tongue_twister_{hash(text)}.mp3")

        # 音声生成
        tts = gTTS(text=text, lang='ja', slow=False)
        temp_file = f"{filename}_temp.mp3"
        tts.save(temp_file)

        # 速度調整
        sound = AudioSegment.from_mp3(temp_file)

        # speedが1.0より大きい場合は早く、小さい場合は遅く
        if speed != 1.0:
            # speedupメソッドを用いて速度を変更
            sound = self._change_speed(sound, speed)

        # 最終的なファイルを保存
        sound.export(filename, format="mp3")

        # 一時ファイルを削除
        if os.path.exists(temp_file):
            os.remove(temp_file)

        print(f"音声ファイルを生成しました: {filename} (速度: {speed}倍)")

        # Colab上で直接再生するが、自動再生はしない
        if play_in_colab:
            try:
                display(Audio(filename, autoplay=False))  # autoplayをFalseに変更
                print("再生ボタンを押して音声を聞くことができます")
            except Exception as e:
                print(f"Colabでの再生に失敗しました: {e}")

        return filename

    def _change_speed(self, sound, speed_factor):
        """音声の速度を変更する"""
        # speedupメソッドを使用する場合（速度を上げる場合）
        if speed_factor > 1.0:
            # speedup関数は内部的にはフレームレートを変更する
            return sound.speedup(playback_speed=speed_factor)
        # 遅くする場合
        elif speed_factor < 1.0:
            # スピードダウンの場合は逆数を取って処理
            return sound._spawn(sound._data, overrides={
                "frame_rate": int(sound.frame_rate * speed_factor)
            }).set_frame_rate(sound.frame_rate)
        return sound

# Google Colab用の実行関数
def run_generator(count=None, generate_audio=False, audio_speed=1.5, play_in_colab=True):
    """早口言葉ジェネレーターを実行する関数"""
    # ジェネレーターインスタンスを作成
    generator = TongueTwisterGenerator()

    # 指定された数の早口言葉を生成
    results = generator.generate(count)

    # 結果を表示
    for i, result in enumerate(results, 1):
        print(f"{i}. {result}")

        # 音声生成オプションがオンの場合
        if generate_audio:
            filename = f"twister_{i}.mp3"
            generator.generate_audio(result, filename, speed=audio_speed, play_in_colab=play_in_colab)
            # 連続再生の場合は少し間を空ける
            if play_in_colab and i < len(results):
                time.sleep(2)  # 2秒待機

# メイン処理部分
if __name__ == "__main__":
    # 説明を表示
    print("=== 早口言葉ジェネレーター ===")
    print(f"生成モード: {GENERATION_MODE}")
    print(f"難易度: {DIFFICULTY_LEVEL}")
    print(f"使用方言: {', '.join(SELECTED_DIALECTS)}")

    # 音声生成機能の案内
    if SPEECH_SETTINGS["enable"]:
        print(f"音声生成: 有効 (速度: {SPEECH_SETTINGS['default_speed']}倍)")
        print(f"Colab再生: {'有効（自動再生なし）' if SPEECH_SETTINGS['play_in_colab'] else '無効'}")
    else:
        print("音声生成: 無効")

    #コピーライト？
    print("")
    print("@Take63")
    print("")

    # 音声生成オプションを設定
    generate_audio = SPEECH_SETTINGS["enable"]
    audio_speed = SPEECH_SETTINGS["default_speed"]
    play_in_colab = SPEECH_SETTINGS["play_in_colab"]

    # ジェネレーターを実行
    run_generator(generate_audio=generate_audio, audio_speed=audio_speed, play_in_colab=play_in_colab)


=== 早口言葉ジェネレーター ===
生成モード: standard
難易度: 普通
使用方言: 博多弁, 関西弁, 津軽弁, 沖縄方言, 東北弁
音声生成: 無効

@Take63

早口言葉バリエーション数: 1031個

=== 早口言葉の組み合わせ数 ===
基本パターン: 388通り
方言パターン: 100通り
音韻パターン: 86通り
行・段パターン: 2526784通り
単語ルール: 270通り
自動生成可能: 486491896通り
理論上の合計: 489019524通り
1. バイオフォトニクス
2. 面発光レーザー
3. コロイド結晶
4. まるまるま
5. バタフライから水泳を楽しむ
6. 雲のような雲が雲のように送る
7. 湯気の呟くで委ねるを持つ
8. コーヒーを飲む
9. トポロジカル超伝導
10. しゅっせきしゅっせきしゅっせきじゅうたんじゅうたん
11. はたはたはたはたはたはたほたるのひかり
12. 麒麟は{adjective}
13. 止むのやちで夜な夜なを買う
14. まみむめもまみむめも
15. だばだば、だばだばだ、だばだばだば
16. 剥き出し、夫婦、見なす
17. 夏至の修道院で象が送る
18. だんだらちょうちょがちょうだらちょうちょ
19. おいおいおちょこおいしゃさん
20. ライブハウスからライブハウスまで進む
21. 非線形動力学
22. クジラはクジラに勝ち、クジラはクジラに勝つ
23. うちなーんちゅ、うちなーんちゅやー、うちなーんちゅうちなー
24. なんぼ、なんぼだべ、なんぼなんぼ
25. 宇宙インフレーション理論
26. 音符ものの音楽を録音する
27. 夢を見る
28. 総理、揃える、公害
29. 知るの仕草で入れるをいる
30. いすずのいすいすずのいすうぐいすうぐいす
31. モジュラー形式
32. 直接的な言葉を伝える
33. 生存、県庁、見解
34. 野球を捕る
35. システム生物学
36. 理解、六角、理髪
37. {adjective}古都で{adjective}中華丼を覚える
38. わたしまけましたわ
39. 春雨ものの春を訪れる
40. 丁寧に公園で決めるなさい
41. 公園を遊ぶ
42. れもんれもんれもんれもんろうそくろうそく
43. 量子ホール効果
44. ラクダが大福を

In [ ]:
import random

# 「あ行」から「わ行」までの定義
kana_groups = {
    "あ行": ["あ", "い", "う", "え", "お"],
    "か行": ["か", "き", "く", "け", "こ"],
    "さ行": ["さ", "し", "す", "せ", "そ"],
    "た行": ["た", "ち", "つ", "て", "と"],
    "な行": ["な", "に", "ぬ", "ね", "の"],
    "は行": ["は", "ひ", "ふ", "へ", "ほ"],
    "ま行": ["ま", "み", "む", "め", "も"],
    "や行": ["や", "ゆ", "よ"],
    "ら行": ["ら", "り", "る", "れ", "ろ"],
    "わ行": ["わ"]
}

# 仮の単語データベース (1〜6文字の単語をランダム生成)
words = ["".join(random.choices("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわ", k=random.randint(2, 6))) for _ in range(100000)]

# 各行ごとに単語を分類
kana_dict = {key: [] for key in kana_groups}

for word in words:
    for key, chars in kana_groups.items():
        if word[0] in chars:
            kana_dict[key].append(f'"{word}"')
            break

# 各行に1500単語ずつ分割してファイル出力
import os
os.makedirs("kana_words", exist_ok=True)

for key, value in kana_dict.items():
    file_path = f'kana_words/{key}_words.txt'
    with open(file_path, 'w', encoding='utf-8') as f:
        chunks = [", ".join(value[i:i+1500]) for i in range(0, len(value), 1500)]
        for chunk in chunks:
            f.write(f'"{key}": [{chunk}]\n')

print("ファイルが生成されました。kana_words フォルダを確認してください。")

ファイルが生成されました。kana_words フォルダを確認してください。


In [ ]:
!pip install gTTS pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
